In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v3/fact/granularity=monthly/ --recursive --human --summarize | tail -5
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-pre-aggr.v3/fact/granularity=yearly/  --recursive --human --summarize | tail -5

# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-07-22/ --recursive
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-11-10/
echo 'aaa'
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/_update_date=2020-11-10/
echo 'bbb'
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.in-unified-product-product-mapping.v1/dimension/_update_date=2020-11-10/

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-07-18 --recursive --human
 
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-07-18 --recursive --human

In [0]:

# 1502056247|  1502056246
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-07-18").where("app_id = 1502056247 ").show()
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-07-19").where("app_id = 1502056247 ").show()
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-07-20").where("app_id = 1502056247 ").show()

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-07-19").select("app_id",
#                                                                                         "company_id",
#                                                                                         "parent_company_id",
#                                                                                         "publisher_id").where("app_id = 698195346 and publisher_id = 1452187002").show()

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-07-17").select("app_id",
                                                                                        "company_id",
                                                                                        "parent_company_id",
                                                                                        "publisher_id").where("app_id = 1502056247").distinct().show()


# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-07-18").select("app_id",
#                                                                                         "company_id",
#                                                                                         "parent_company_id",
#                                                                                         "publisher_id").where("app_id = 1502056247").distinct().show()

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-07-18").where("app_id = 1502056247").distinct().show()


In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/").where("app_id = 1502056247").distinct().show()
store_data = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/"
df1 = spark.read.format('delta').load(store_data).where("granularity='daily' and date between '2020-07-18' and '2020-07-18' ")
df1.show()


In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-07-18").select("app_id",
                                                                                        "company_id",
                                                                                        "parent_company_id",
                                                                                        "publisher_id").where("app_id = 698195346").show()


In [0]:

store_monthly_data = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-pre-aggr.v3/fact/"
df1 = spark.read.format('delta').load(store_monthly_data).where("granularity='yearly'").cache()

df1.select("app_id").createOrReplaceTempView("store_app_id")


df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-06-29/").cache()
df2.select("app_id").createOrReplaceTempView("dna_app_id")


spark.sql("select count(1) from ( select distinct app_id from store_app_id except select distinct app_id from dna_app_id ) as prod").show(300)




In [0]:

import datetime

start = "2010-08-01"
end = "2020-06-21"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)



def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)



month = list(set([datetime.datetime.strftime(last_day_of_month(datetime.datetime.strptime(x, '%Y-%m-%d')), '%Y-%m-%d') for x in dates ]))
month.sort()


test_month_start_end_date = [ (x[:7]+"-01", x) for x in month]

print test_month_start_end_date

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/publisher_id/


In [0]:

from pyspark.sql import functions as F

import datetime

# generate data
# for day in test_month_start_end_date:
#     publisher_data = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher.v1/fact/"
#     df1 = spark.read.format('delta').load(publisher_data).where("granularity='daily' and date between '{}' and '{}'".format(day[0], day[1])).select("publisher_id").withColumn("date", F.lit(day[0]))
#     df1.createOrReplaceTempView("store_publisher_id")
#     df1.coalesce(5).write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/publisher_id/",
#                                               mode="append",
#                                               partitionBy=["date"])



df_publisher = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/publisher_id/")
df_publisher.createOrReplaceTempView("top_publisher_id")

df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-06-29/").cache()
df2.select("publisher_id").createOrReplaceTempView("dna_publisher_id")



publisher_not_exist_list = spark.sql("select distinct publisher_id from top_publisher_id except select distinct publisher_id from dna_publisher_id").collect()
# spark.sql("select distinct publisher_id from top_publisher_id except select distinct publisher_id from dna_publisher_id").show(200)

# spark.sql("select count(1) from ( select distinct publisher_id from top_publisher_id except select distinct publisher_id from dna_publisher_id ) as prod").show()




In [0]:

result_list = list()
for x in test_month_start_end_date:
    print x[0]
    path= "s3://b2c-prod-data-pipeline-qa/aa.store/publisher_id/date={}/".format(str(x[0]))
    print path
    df_publisher = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-qa/aa.store/publisher_id/").parquet(path)

    df_publisher.createOrReplaceTempView("top_publisher_id")
    publisher_not_in_with_date = spark.sql("select * from top_publisher_id where publisher_id in ({})".format(",".join(str_publisher_not_in_list))).collect()
    result_list.extend([x[0],publisher_not_in_with_date])


In [0]:

print len(result_list)
#print result_list[229][0]
for y in range(231,238,2):
    print len(set([x[0] for x in result_list[y]]))

In [0]:

print [y for y in range(1,325,2)]

In [0]:

publisher_data = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher.v1/fact/"
df1 = spark.read.format('delta').load(publisher_data).where("granularity='daily' and date between '2020-02-01' and '2020-02-29' and publisher_id in (1498066528, 1498416097, 1498034787, 1499200100, 20200003048421, 1498448097, 1497678508, 1499259050, 1494487883, 1495587073, 1499476594, 1499199631, 1499207728, 1498972434, 1499265907, 1498947191, 1498180338, 1217928316) and device_code !='ios-all' ")
df1.orderBy("free_app_download", ascending=False).show()


In [0]:

str_publisher_not_in_list = [str(x) for x in publisher_not_in_list]
# print ",".join(str_publisher_not_in_list)

In [0]:


publisher_not_in_list = [289679193, 474724068, 474737980, 551839768, 569409181, 577073127, 610955645, 636421687, 684892946, 686449807, 715441541, 721911537, 744070884, 866617777, 919745844, 1003510429, 1026901472, 1104191752, 1157181090, 20200000009498, 20200000010511, 20200000012480, 20200000014856, 20200000026105, 20200000029480, 20200000033257, 20200000045088, 20200000100621, 20200000103073, 20200000112553, 20200000118166, 20200000123559, 20200000128853, 20200000178148, 20200000199472, 20200000202915, 20200000254298, 20200000255537, 20200000262672, 20200000268743, 20200000271288, 20200000361347, 20200000366726, 20200000381910, 20200000385183, 20200000388860, 20200000403464, 20200000409401, 20200000419767, 20200000419842, 20200000443452, 20200000446753, 20200000450277, 20200000462757, 20200001888376, 363499736, 411206394, 456960796, 471359320, 529826126, 535557023, 549802399, 668311596, 711564469, 935577462, 969934912, 20200000004097, 20200000006307, 20200000022199, 20200000025646, 20200000027532, 20200000061516, 20200000119525, 20200000130198, 20200000136216, 20200000158496, 20200000165858, 20200000192550, 20200000193154, 20200000193981, 20200000201583, 20200000203498, 20200000214816, 20200000216831, 20200000252968, 20200000279259, 20200000354013, 20200000360636, 20200000365816, 20200000379297, 20200000383001, 20200000383483, 20200000383858, 20200000413892, 20200000422641, 20200000424655, 20200000436017, 20200000437267, 20200000441734, 20200000446672, 20200000457355, 20200001932090, 321506742, 465899255, 483272548, 563797954, 662923595, 691307157, 934417336, 1030902626, 1041260001, 1047246341, 1117369948, 1166922945, 20200000001338, 20200000037336, 20200000039764, 20200000045061, 20200000070048, 20200000072415, 20200000095252, 20200000107763, 20200000125887, 20200000151276, 20200000156396, 20200000170187, 20200000188355, 20200000197525, 20200000201960, 20200000239282, 20200000258476, 20200000262936, 20200000281120, 20200000351560, 20200000357961, 20200000362203, 20200000366816, 20200000383439, 20200000384608, 20200000391948, 20200000400120, 20200000409157, 20200000413397, 20200000414370, 20200000425498, 20200000431747, 20200000432019, 20200000443988, 20200000447138, 20200000458015, 20200002175819, 303076295, 323241118, 367706767, 367903856, 384490807, 407413403, 458813562, 581914455, 604105154, 785537179, 915074326, 918609651, 941528148, 996996718, 1009442510, 1058528141, 1059247542, 1178454068, 20200000012557, 20200000050344, 20200000068088, 20200000084727, 20200000091048, 20200000134981, 20200000213710, 20200000214456, 20200000217926, 20200000240205, 20200000244204, 20200000366391, 20200000366724, 20200000385200, 20200000404301, 20200000436128, 20200000438096, 20200000442774, 20200000457538, 20200001872049, 293170168, 342128086, 364234221, 364576097, 427480051, 546287349, 562519824, 568341812, 577105209, 781863128, 939523213, 963637612, 1114832719, 1326620428, 20200000000158, 20200000039251, 20200000111700, 20200000125517, 20200000153435, 20200000163216, 20200000163796, 20200000174763, 20200000202983, 20200000219806, 20200000238785, 20200000249644, 20200000252676, 20200000255893, 20200000258378, 20200000274960, 20200000276338, 20200000367649, 20200000393988, 20200000397809, 20200000401898, 20200000424469, 20200000427213, 20200000433185, 20200000434100, 20200000438636, 20200000441505, 20200000446820, 20200000452510, 20200000461320, 20200000463544, 344156885, 388173036, 441953397, 449118865, 450795513, 549990577, 597517751, 650684932, 817372460, 823518885, 827186281, 828578246, 883078087, 972019427, 985803004, 1106868332, 1118745547, 1151646674, 1177068706, 1297825524, 20200000011205, 20200000020956, 20200000022341, 20200000061858, 20200000071013, 20200000121982, 20200000161493, 20200000162380, 20200000167543, 20200000175723, 20200000184479, 20200000195287, 20200000216192, 20200000216338, 20200000224588, 20200000237548, 20200000241023, 20200000249875, 20200000264119, 20200000264904, 20200000276532, 20200000280241, 20200000355974, 20200000360869, 20200000377736, 20200000381654, 20200000384010, 20200000393919, 20200000410211, 20200000411503, 20200000414438, 20200000422738, 20200000445483, 20200000454024, 20200000458342, 20200000463940, 20200002102277, 305343404, 374211477, 388467884, 393335869, 441424603, 575147386, 595361888, 602065803, 627521020, 714903632, 887343320, 901500107, 942691321, 950594073, 1027230451, 1042644568, 1059405547, 1084154030, 1116026071, 1174241481, 20200000010820, 20200000026185, 20200000034344, 20200000043781, 20200000056960, 20200000059536, 20200000067614, 20200000098357, 20200000116706, 20200000165919, 20200000168164, 20200000176324, 20200000187177, 20200000210522, 20200000218424, 20200000229472, 20200000244138, 20200000255105, 20200000276351, 20200000281358, 20200000359348, 20200000376496, 20200000377556, 20200000384963, 20200000414001, 20200000421129, 20200000430445, 20200000434490, 20200000442997, 20200000455635, 20200000457327, 20200000459238, 20200000463507, 300590611, 392761527, 405743220, 471832344, 493233875, 513474544, 545991711, 553604684, 589779009, 739732075, 776289884, 900404152, 949701151, 969434310, 981305941, 987953868, 1020936262, 1037938081, 1128488517, 1445967370, 20200000000117, 20200000028118, 20200000057369, 20200000086207, 20200000089988, 20200000100933, 20200000116730, 20200000127362, 20200000164208, 20200000172717, 20200000202492, 20200000212976, 20200000214714, 20200000217508, 20200000222245, 20200000255191, 20200000363179, 20200000377660, 20200000383287, 20200000389348, 20200000390477, 20200000417989, 20200000423440, 20200000429543, 20200000429994, 20200000430213, 20200000439839, 20200000451878, 20200000454003, 20200000553202, 20200001992077, 20200002573514, 306572986, 323470584, 324684580, 339748933, 441919897, 443637419, 464832656, 561941526, 577232024, 577491499, 636511657, 714797806, 816347839, 869231055, 885989000, 943869618, 1003723418, 1007143930, 1041873285, 1075979610, 1100180138, 1149340866, 1150901618, 20200000000867, 20200000009593, 20200000012204, 20200000025403, 20200000051566, 20200000068798, 20200000070920, 20200000082447, 20200000094674, 20200000105727, 20200000133635, 20200000137769, 20200000146996, 20200000154281, 20200000166488, 20200000171947, 20200000184206, 20200000186332, 20200000192779, 20200000221572, 20200000230315, 20200000242455, 20200000272297, 20200000282780, 20200000356072, 20200000361674, 20200000366156, 20200000407677, 20200000417967, 20200000423333, 20200000435896, 20200000438624, 20200000449580, 300704847, 364295267, 382552638, 530734732, 541103737, 634444186, 650645305, 820391809, 865209835, 918054748, 940452319, 995999703, 1039141908, 1082610285, 1083732797, 1112807153, 1132517788, 1163677172, 1209914348, 1435800576, 20200000050295, 20200000062655, 20200000081123, 20200000127427, 20200000139498, 20200000150521, 20200000172669, 20200000188426, 20200000203192, 20200000226791, 20200000273258, 20200000367578, 20200000376711, 20200000377477, 20200000396033, 20200000398967, 20200000409358, 20200000414198, 20200000422480, 20200000429420, 20200000438062, 20200000441514, 20200000456274, 20200002302839, 330803072, 448278467, 464339272, 472963415, 568438534, 570319431, 584362474, 598223944, 669196929, 677464631, 681885391, 777981129, 788570313, 953522237, 967422975, 969435065, 978759625, 995060131, 1084651716, 1140088345, 1164564053, 1401252130, 20200000009033, 20200000009966, 20200000029982, 20200000033991, 20200000038672, 20200000049601, 20200000056526, 20200000059945, 20200000075200, 20200000092590, 20200000097234, 20200000106135, 20200000138972, 20200000148586, 20200000155358, 20200000165698, 20200000170122, 20200000183384, 20200000188908, 20200000211070, 20200000225811, 20200000229614, 20200000242094, 20200000248408, 20200000251899, 20200000282574, 20200000354232, 20200000355440, 20200000357045, 20200000403549, 20200000437458, 20200000441313, 20200000443286, 20200000444692, 20200000447160, 382509315, 392788790, 444435600, 465213808, 480805652, 495281928, 499541243, 547566699, 553573601, 618394250, 632012899, 716238013, 786185899, 898968647, 907174449, 909170492, 989148225, 1017492454, 1136936929, 1181616006, 20200000014411, 20200000015191, 20200000043922, 20200000056806, 20200000078330, 20200000082934, 20200000107775, 20200000120704, 20200000125625, 20200000129304, 20200000149854, 20200000157380, 20200000171046, 20200000186181, 20200000190521, 20200000223071, 20200000267943, 20200000275910, 20200000277124, 20200000282675, 20200000366829, 20200000385582, 20200000413725, 20200000425818, 20200000434574, 20200000438064, 20200000439283, 20200000444636, 20200000448669, 20200000461348, 20200000637198, 20200001388253, 314780738, 364596328, 429844857, 446324234, 483181998, 492050035, 497933996, 583066190, 619125868, 638853147, 645949180, 767432918, 794866182, 804641004, 912852888, 922609543, 928780909, 948255632, 965842202, 989804926, 1006275482, 1048130311, 1135354652, 1136617049, 1187617300, 20200000074957, 20200000085967, 20200000087670, 20200000131027, 20200000167173, 20200000195303, 20200000198575, 20200000233218, 20200000240072, 20200000246017, 20200000253799, 20200000264146, 20200000266961, 20200000271304, 20200000359277, 20200000359320, 20200000362334, 20200000383060, 20200000383699, 20200000395769, 20200000400153, 20200000410198, 20200000419105, 20200000445631, 20200000463803, 20200001603915, 383359044, 401710533, 409098342, 435913585, 476334096, 509458284, 512651258, 521903725, 606190854, 632079234, 647132041, 656971078, 852287972, 884495525, 940020359, 990905312, 1038885254, 1073000685, 1090927279, 1110349058, 1114886979, 1147119609, 20200000006501, 20200000046582, 20200000059404, 20200000064150, 20200000066794, 20200000069882, 20200000128861, 20200000137839, 20200000155662, 20200000158586, 20200000161120, 20200000161722, 20200000165235, 20200000182484, 20200000185954, 20200000190793, 20200000195224, 20200000206670, 20200000206703, 20200000231162, 20200000258088, 20200000279124, 20200000280633, 20200000280722, 20200000384772, 20200000385209, 20200000390443, 20200000408974, 20200000435969, 20200000440000, 20200000447357, 20200000463252, 20200001761918, 316050001, 376178732, 416788089, 427355854, 478451500, 499195376, 593448569, 609845786, 714508224, 917286208, 946789062, 1180885627, 1181904201, 20200000011609, 20200000059250, 20200000059265, 20200000094853, 20200000098823, 20200000121081, 20200000139164, 20200000145638, 20200000149644, 20200000151070, 20200000192872, 20200000218586, 20200000227998, 20200000256941, 20200000268421, 20200000275946, 20200000276425, 20200000277099, 20200000370415, 20200000372841, 20200000404957, 20200000419358, 20200000429434, 20200000432761, 20200000432768, 20200000434330, 20200000434556, 20200000437045, 20200000444021, 20200000460694, 20200001648811, 341311361, 353386972, 427543815, 476937430, 485470008, 493226494, 533173905, 535609266, 555565328, 622903822, 640468746, 824289097, 881404173, 971233157, 1063128473, 20200000013862, 20200000016016, 20200000023580, 20200000046521, 20200000054140, 20200000069502, 20200000079656, 20200000086286, 20200000087819, 20200000103798, 20200000111999, 20200000119061, 20200000134977, 20200000150330, 20200000186637, 20200000206091, 20200000217060, 20200000255309, 20200000270261, 20200000282472, 20200000384385, 20200000384452, 20200000391261, 20200000396194, 20200000404988, 20200000408609, 20200000411526, 20200000422227, 20200002500276, 531010298, 575215183, 579825429, 641679213, 648030052, 650469864, 666959376, 672710052, 705203903, 796726213, 924503619, 963382371, 976689529, 981198574, 1006408029, 1024745636, 1033387365, 1088404293, 1369352616, 1502947636, 20200000003241, 20200000026348, 20200000034708, 20200000061368, 20200000067363, 20200000087768, 20200000105379, 20200000111230, 20200000117398, 20200000143506, 20200000145797, 20200000163622, 20200000176939, 20200000177047, 20200000177271, 20200000202158, 20200000204812, 20200000206348, 20200000239927, 20200000259342, 20200000270532, 20200000270704, 20200000275024, 20200000383265, 20200000402988, 20200000406428, 20200000422160, 20200000423529, 20200000426351, 20200000429529, 20200000440802, 20200000441466, 20200000459610, 20200000460876, 20200002327391, 381342267, 383588066, 406750496, 420009108, 483178257, 526012271, 550876711, 609011789, 654810212, 687287652, 743031884, 879675363, 909349642, 933397897, 949275594, 952673792, 971756507, 994133779, 1053012308, 1077600556, 1093979561, 1117783372, 1220835300, 1358277828, 1425700165, 1498066528, 20200000019634, 20200000024632, 20200000029928, 20200000074665, 20200000100535, 20200000101657, 20200000102470, 20200000122518, 20200000150798, 20200000169853, 20200000180314, 20200000200777, 20200000205225, 20200000214831, 20200000215597, 20200000221119, 20200000221754, 20200000227720, 20200000230051, 20200000243938, 20200000274753, 20200000350274, 20200000353920, 20200000369771, 20200000398250, 20200000399394, 20200000405711, 20200000407908, 20200000411620, 20200000429818, 20200000431117, 20200000441171, 20200000461187, 20200000462780, 331177703, 377625831, 392502056, 406388984, 492495415, 515884174, 529652920, 604897665, 634299627, 739758339, 925201227, 940702381, 957152282, 972400169, 1007204523, 1023681711, 1027270126, 1076700546, 1136200984, 1183772149, 1451167348, 20200000014306, 20200000018849, 20200000027633, 20200000092111, 20200000107058, 20200000196112, 20200000213553, 20200000235429, 20200000252657, 20200000257984, 20200000264580, 20200000267574, 20200000278732, 20200000281311, 20200000385025, 20200000388589, 20200000394165, 20200000394870, 20200000400721, 20200000401144, 20200000402648, 20200000410570, 20200000415631, 20200000462606, 20200003165524, 322681972, 338365297, 347415188, 360593530, 363360636, 367292101, 371808484, 412591849, 449735650, 464837515, 466541471, 488627858, 514345557, 534477679, 547726555, 596402997, 662611545, 714298477, 717427889, 730295038, 782426076, 849796318, 900313219, 967327312, 1001244846, 1047961826, 1062771297, 1112068171, 1498972434, 20200000042335, 20200000088646, 20200000132743, 20200000173347, 20200000187261, 20200000214764, 20200000225859, 20200000261066, 20200000280051, 20200000359646, 20200000372090, 20200000382707, 20200000386578, 20200000390302, 20200000390489, 20200000391829, 20200000403255, 20200000409161, 20200000416622, 20200000418251, 20200000421948, 20200000444106, 20200000452671, 20200000455385, 20200000612014, 347800756, 455210120, 463469532, 467923185, 502458878, 541693525, 590330669, 624512118, 641900855, 744966613, 918035581, 929346489, 949737354, 975985952, 979092836, 1052704329, 1131519629, 1159035153, 20200000021660, 20200000026334, 20200000058146, 20200000085346, 20200000091828, 20200000095829, 20200000125476, 20200000170155, 20200000179899, 20200000183211, 20200000186857, 20200000219960, 20200000222716, 20200000226432, 20200000250930, 20200000356968, 20200000365974, 20200000387870, 20200000394545, 20200000403597, 20200000428882, 20200000440584, 20200000443587, 20200000444398, 20200000453077, 20200000456907, 20200000458642, 20200001887622, 20200003132478, 291322250, 379852725, 398165195, 405634168, 409823859, 471347413, 486781045, 547457727, 605676336, 617098629, 624026187, 687869470, 857690884, 891904928, 896469937, 899583583, 1019225095, 1057961675, 1077696526, 1079335042, 1123795045, 1165725935, 20200000049027, 20200000054152, 20200000067804, 20200000074994, 20200000075692, 20200000093494, 20200000115404, 20200000119004, 20200000130760, 20200000135863, 20200000156471, 20200000162687, 20200000178894, 20200000194075, 20200000213889, 20200000221787, 20200000228259, 20200000249842, 20200000258259, 20200000353810, 20200000367518, 20200000368857, 20200000369621, 20200000385704, 20200000391093, 20200000394579, 20200000394634, 20200000411680, 20200000424096, 20200000428138, 20200000453473, 20200000462774, 20200002382943, 312266675, 331426518, 350361202, 388467713, 447331712, 488748114, 506462353, 520284219, 529170920, 571901513, 657682031, 775147680, 881981094, 896143616, 950876153, 1074470421, 1091215595, 1171369240, 1237458603, 1382926228, 20200000005837, 20200000046835, 20200000048899, 20200000055052, 20200000058921, 20200000075540, 20200000082417, 20200000116249, 20200000130173, 20200000130841, 20200000159603, 20200000168963, 20200000190185, 20200000205208, 20200000238954, 20200000260546, 20200000261134, 20200000275244, 20200000361690, 20200000362748, 20200000371200, 20200000373423, 20200000390938, 20200000391520, 20200000400410, 20200000413611, 20200000418086, 20200000422028, 20200000425078, 20200000434380, 20200000447156, 20200000457654, 20200000458614, 20200001479897, 20200002552338, 373903654, 527197872, 575684686, 588358613, 592986394, 654737072, 845144356, 898878412, 922394137, 988842678, 1071361320, 1118431695, 1136744871, 1186827808, 20200000021509, 20200000029505, 20200000049059, 20200000051635, 20200000068666, 20200000080215, 20200000090813, 20200000108874, 20200000132234, 20200000147522, 20200000165387, 20200000176001, 20200000199135, 20200000218785, 20200000220019, 20200000255999, 20200000271959, 20200000381843, 20200000383819, 20200000395869, 20200000400524, 20200000405457, 20200000419830, 20200000421935, 20200000424797, 20200000460606, 20200001717501, 20200001883800, 20200002126532, 20200003236033, 366703930, 448639966, 475336263, 503092422, 542557212, 626148774, 626696483, 629042352, 727136848, 738466189, 758871886, 881613536, 903019028, 1220304431, 20200000013285, 20200000026683, 20200000044581, 20200000064995, 20200000076173, 20200000082021, 20200000088570, 20200000092411, 20200000109549, 20200000110727, 20200000115916, 20200000118682, 20200000144335, 20200000161770, 20200000183453, 20200000184716, 20200000199608, 20200000215742, 20200000233374, 20200000237209, 20200000248710, 20200000251530, 20200000256039, 20200000355347, 20200000386267, 20200000395828, 20200000402561, 20200000410935, 20200000443571, 20200000457568, 20200003183493, 299616801, 331291960, 371817955, 376535806, 410395246, 493864839, 496052481, 500962489, 542572296, 553823952, 669297049, 695120488, 742625884, 789856332, 842842640, 868289602, 890390751, 894918913, 920160643, 931228349, 932441611, 953006428, 1073949287, 1095231372, 1109008912, 1114422540, 1321641628, 20200000001539, 20200000037035, 20200000111196, 20200000125110, 20200000139395, 20200000155582, 20200000157370, 20200000160501, 20200000166454, 20200000193914, 20200000199874, 20200000203782, 20200000260071, 20200000263288, 20200000282810, 20200000354200, 20200000361020, 20200000366996, 20200000370124, 20200000404865, 20200000406878, 20200000421259, 20200000425256, 20200000427370, 20200000438302, 20200000438524, 20200002913561, 388540506, 395627741, 399648212, 426907035, 451287325, 469231420, 505446332, 563745449, 586076398, 668576857, 721567545, 737006024, 839703707, 863114253, 875269585, 943432325, 975873718, 1030150970, 1042519200, 1074628617, 1112905847, 1149921086, 1154900079, 1238978318, 20200000006252, 20200000014921, 20200000061632, 20200000107551, 20200000146429, 20200000167095, 20200000204891, 20200000225866, 20200000280509, 20200000282917, 20200000352843, 20200000354949, 20200000356940, 20200000357671, 20200000372015, 20200000393804, 20200000414866, 20200000420775, 20200000437434, 20200000442477, 20200000450911, 20200003183522, 354902315, 415038787, 438208049, 562184187, 623592465, 681283192, 687877464, 715668236, 843577241, 891597068, 905425870, 951391381, 969686618, 980232807, 992896211, 1080487957, 1085105198, 1091308524, 1126471763, 1128075013, 1275410504, 20200000004340, 20200000026614, 20200000039784, 20200000041765, 20200000058203, 20200000105792, 20200000119232, 20200000124670, 20200000133599, 20200000135632, 20200000138523, 20200000148624, 20200000177995, 20200000181858, 20200000216082, 20200000221306, 20200000228852, 20200000232105, 20200000233645, 20200000240674, 20200000259433, 20200000354601, 20200000360699, 20200000369568, 20200000370832, 20200000381882, 20200000383412, 20200000393963, 20200000405097, 20200000416409, 20200000417442, 20200000438655, 20200000445155, 20200000452168, 20200000460707, 20200000763402, 20200001858531, 20200002618408, 396306670, 438208317, 442522082, 522069155, 571649398, 578683298, 660191002, 686365571, 708986315, 843267350, 915469477, 916745147, 939434515, 964743113, 980271293, 1029643802, 1123132173, 1141518627, 1350714008, 1472826712, 20200000025837, 20200000040993, 20200000051606, 20200000085434, 20200000124678, 20200000125588, 20200000146802, 20200000170461, 20200000171425, 20200000173112, 20200000174179, 20200000184010, 20200000196540, 20200000202217, 20200000206330, 20200000226581, 20200000234040, 20200000243658, 20200000244756, 20200000249775, 20200000264862, 20200000267111, 20200000358854, 20200000369903, 20200000371951, 20200000377046, 20200000377554, 20200000386196, 20200000389338, 20200000429244, 20200000441206, 20200000450762, 20200000451016, 20200000456335, 20200002150667, 297606951, 348737462, 381088017, 406136981, 436582479, 451058996, 504677517, 555194756, 581731651, 602990197, 611436052, 668598022, 687291274, 854356557, 932191687, 934844916, 1063872409, 1091512762, 1093415242, 1153420249, 20200000005469, 20200000008638, 20200000013680, 20200000035118, 20200000037633, 20200000064599, 20200000066326, 20200000118765, 20200000131509, 20200000135127, 20200000158791, 20200000166267, 20200000189545, 20200000198287, 20200000214613, 20200000230238, 20200000260442, 20200000266294, 20200000270976, 20200000272431, 20200000282265, 20200000351812, 20200000359115, 20200000386500, 20200000401301, 20200000401937, 20200000407028, 20200000408321, 20200000411241, 20200000415422, 20200000443617, 20200000449723, 20200000450359, 330272420, 386832667, 551798799, 664422011, 695094158, 708196645, 721651959, 731868291, 973303004, 973895042, 1321250121, 1395808031, 20200000005829, 20200000005992, 20200000029811, 20200000051636, 20200000083123, 20200000124821, 20200000145429, 20200000147414, 20200000157667, 20200000171641, 20200000179665, 20200000187182, 20200000192582, 20200000204062, 20200000207395, 20200000207907, 20200000217786, 20200000257845, 20200000279014, 20200000352299, 20200000354838, 20200000357449, 20200000374190, 20200000383229, 20200000389345, 20200000398115, 20200000414527, 20200000421843, 20200000424225, 20200000424409, 20200000437041, 20200000437599, 20200000438692, 20200000441237, 20200000443941, 20200000449885, 20200000452940, 20200000593415, 341040894, 361285480, 364297166, 409794234, 455072135, 479904046, 492477901, 576442111, 577492451, 672828590, 696678010, 727420266, 739699649, 855102893, 912536422, 933462805, 1023681159, 1138938804, 1164046790, 1474259319, 20200000017196, 20200000027898, 20200000039523, 20200000046866, 20200000063437, 20200000066023, 20200000071524, 20200000081607, 20200000085424, 20200000088540, 20200000108629, 20200000110165, 20200000118230, 20200000138137, 20200000160457, 20200000167346, 20200000172241, 20200000179365, 20200000189781, 20200000196295, 20200000217065, 20200000220263, 20200000232315, 20200000233378, 20200000236295, 20200000236616, 20200000239280, 20200000276151, 20200000350796, 20200000366441, 20200000376531, 20200000386102, 20200000413885, 20200000438711, 20200002160405, 20200002518868, 381770966, 412827941, 457835054, 495602542, 533800725, 554936942, 566652732, 675102189, 951704333, 1053512688, 1073510906, 1093699303, 1108964305, 1128259891, 1142261822, 1177516437, 20200000034891, 20200000046066, 20200000052365, 20200000062085, 20200000084635, 20200000089055, 20200000093006, 20200000106407, 20200000159525, 20200000163870, 20200000223274, 20200000239902, 20200000255508, 20200000350606, 20200000358099, 20200000368631, 20200000381507, 20200000413995, 20200000415158, 20200000417295, 20200000418455, 20200000421988, 20200000422037, 20200000425731, 20200000425877, 20200000428338, 20200000430843, 20200000439521, 20200000448666, 20200000448739, 20200000461782, 20200000462949, 20200002963433, 367816156, 374151636, 387232375, 396541253, 401626263, 450697657, 509547477, 523005640, 579933458, 621478260, 669216174, 930026670, 931943412, 935566920, 1035263816, 1092463295, 1182962353, 1212117595, 20200000005242, 20200000009012, 20200000033564, 20200000041298, 20200000079933, 20200000093401, 20200000113436, 20200000126272, 20200000135763, 20200000138698, 20200000143589, 20200000145317, 20200000164098, 20200000191485, 20200000230110, 20200000240204, 20200000249301, 20200000257566, 20200000393362, 20200000397582, 20200000408952, 20200000417927, 20200000418834, 20200000427000, 20200000439543, 20200000439573, 20200000450794, 20200000461881, 20200000461894, 20200001461134, 364388950, 389636848, 404086528, 499757899, 594802437, 893194736, 916281743, 919085364, 943632624, 971215921, 981420466, 989786102, 1007720559, 1016780285, 1058859697, 1118441024, 1212230849, 20200000008451, 20200000010628, 20200000014287, 20200000045138, 20200000045474, 20200000061390, 20200000077026, 20200000112691, 20200000128436, 20200000136738, 20200000147185, 20200000218168, 20200000218607, 20200000224850, 20200000234034, 20200000236087, 20200000276251, 20200000280752, 20200000283388, 20200000352462, 20200000356861, 20200000361382, 20200000365656, 20200000394733, 20200000404801, 20200000406099, 20200000420348, 20200000425064, 20200000437425, 20200000447304, 20200000459280, 20200000461775, 20200002273274, 20200003143112, 375692822, 430807236, 434624320, 462163559, 510173686, 685675706, 696565994, 777317826, 976173170, 1041528293, 1086937448, 1124779414, 1176160546, 1177409041, 1181475133, 20200000009889, 20200000031173, 20200000031972, 20200000052331, 20200000090620, 20200000102396, 20200000107288, 20200000116274, 20200000156768, 20200000164432, 20200000184238, 20200000202420, 20200000225219, 20200000245820, 20200000253140, 20200000268632, 20200000353481, 20200000375321, 20200000377548, 20200000397030, 20200000410784, 20200000417030, 20200000422471, 20200000425804, 20200000432947, 20200000436137, 20200000456389, 20200002153849, 357421934, 362142745, 406419217, 414748665, 504966374, 508667882, 514917848, 524127934, 561416817, 569631758, 570930887, 632016921, 703227139, 720277796, 778658393, 890382703, 904481505, 931267524, 936971630, 966810173, 972657175, 989335709, 1003876872, 1045903703, 1073698593, 1152656624, 20200000013632, 20200000026869, 20200000052040, 20200000062242, 20200000070922, 20200000082409, 20200000099336, 20200000122977, 20200000149895, 20200000153309, 20200000160669, 20200000178332, 20200000226505, 20200000235859, 20200000245709, 20200000256101, 20200000263825, 20200000265603, 20200000283599, 20200000356196, 20200000379257, 20200000380626, 20200000388174, 20200000388176, 20200000391081, 20200000391917, 20200000392156, 20200000395997, 20200000400563, 20200000417835, 20200000420308, 20200000424105, 20200000430059, 20200000431671, 20200000433167, 20200000437004, 20200000437429, 20200000439826, 20200000441158, 20200000441672, 20200000442633, 20200000448662, 20200001674045, 20200001793855, 286046305, 325296453, 373750924, 398129933, 429047995, 582417365, 733773825, 741183306, 808296431, 829212088, 863844475, 924017911, 945643599, 957306947, 966434447, 1119492168, 1186281122, 1244488390, 20200000028392, 20200000045928, 20200000051787, 20200000051949, 20200000070696, 20200000101128, 20200000106788, 20200000118284, 20200000120400, 20200000145882, 20200000151486, 20200000169054, 20200000175435, 20200000180288, 20200000213755, 20200000236129, 20200000267841, 20200000274014, 20200000359863, 20200000360318, 20200000380685, 20200000383177, 20200000403821, 20200000408358, 20200000409854, 20200000419871, 20200000432640, 20200000439810, 393735930, 419227089, 491944739, 496715557, 501892510, 511332635, 561643327, 573116090, 584623206, 585159615, 754054884, 771989093, 850417475, 897446215, 907074008, 908748733, 991262592, 1012298403, 1079974548, 1168914812, 1184819134, 1347569686, 20200000008313, 20200000008938, 20200000034972, 20200000083792, 20200000094704, 20200000096969, 20200000116715, 20200000127918, 20200000144478, 20200000159295, 20200000171259, 20200000175886, 20200000197262, 20200000211350, 20200000212877, 20200000232187, 20200000234087, 20200000240466, 20200000243377, 20200000276681, 20200000280775, 20200000370148, 20200000382231, 20200000420813, 20200000431140, 20200000431777, 20200000451384, 20200000452387, 20200000453390, 20200000461315, 20200000461402, 20200001872257, 327116713, 348094440, 414053537, 421216878, 482365195, 503190232, 635826704, 709189016, 758453889, 834406254, 834894503, 887729939, 917554783, 922558758, 1027101049, 1044318857, 1047733390, 1082956994, 1086978383, 1113459333, 1114598489, 1185942118, 1498034787, 20200000018109, 20200000037451, 20200000061891, 20200000063138, 20200000063991, 20200000066093, 20200000070080, 20200000089481, 20200000092155, 20200000101679, 20200000103769, 20200000107397, 20200000148941, 20200000171262, 20200000196140, 20200000223570, 20200000223985, 20200000248746, 20200000264882, 20200000352545, 20200000353271, 20200000393507, 20200000400358, 20200000406725, 20200000410487, 20200000429010, 20200000433891, 20200000440922, 20200000441519, 20200000449096, 20200000450199, 20200000451113, 20200003073342, 307459878, 361555974, 509546027, 515292220, 610391947, 641859963, 711455226, 815071798, 834315918, 878042995, 906743451, 949205941, 1010704842, 1071310449, 1076239211, 1088641464, 1299623328, 20200000005651, 20200000035335, 20200000056167, 20200000061531, 20200000068604, 20200000082762, 20200000094815, 20200000099206, 20200000126140, 20200000144620, 20200000156414, 20200000156457, 20200000174413, 20200000189399, 20200000194009, 20200000203307, 20200000215616, 20200000230057, 20200000236022, 20200000244283, 20200000257184, 20200000262148, 20200000350504, 20200000361228, 20200000392228, 20200000401517, 20200000403102, 20200000411550, 20200000426092, 20200000456053, 20200000456393, 20200000463239, 287455446, 320636131, 323229106, 336477530, 369610590, 490988442, 765224318, 863040953, 881106329, 1081835031, 1118247592, 1165315677, 20200000030292, 20200000034042, 20200000040575, 20200000057273, 20200000067394, 20200000076015, 20200000078968, 20200000096416, 20200000101600, 20200000130785, 20200000151517, 20200000258174, 20200000279332, 20200000373561, 20200000377948, 20200000384766, 20200000395576, 20200000396507, 20200000407109, 20200000407368, 20200000412954, 20200000442779, 20200000442848, 20200000448322, 20200000449384, 20200000462668, 20200001727340, 330247123, 346776596, 364355046, 369888397, 441129541, 458535809, 671867510, 736179781, 757846905, 839537579, 890757970, 956622914, 1089048531, 20200000020261, 20200000057355, 20200000066506, 20200000069526, 20200000081125, 20200000106549, 20200000109613, 20200000113292, 20200000122114, 20200000138640, 20200000157784, 20200000168188, 20200000168557, 20200000171608, 20200000178993, 20200000200568, 20200000241713, 20200000250702, 20200000261965, 20200000271127, 20200000280237, 20200000359041, 20200000359452, 20200000368194, 20200000374326, 20200000435416, 20200000438250, 20200000441049, 286616280, 396764341, 413419662, 507288895, 537802071, 547167853, 558494861, 712367105, 718829098, 727296976, 796352563, 806223188, 937737946, 1090986940, 1181347321, 1188011548, 1409761777, 20200000011991, 20200000015086, 20200000021327, 20200000042251, 20200000071338, 20200000085554, 20200000100467, 20200000101570, 20200000141978, 20200000176711, 20200000221435, 20200000225289, 20200000229564, 20200000242164, 20200000245498, 20200000247539, 20200000262419, 20200000351376, 20200000354598, 20200000357595, 20200000359714, 20200000378247, 20200000392153, 20200000396313, 20200000398504, 20200000403660, 20200000425870, 20200000425993, 20200000432188, 20200000434608, 20200000447682, 20200000457052, 20200000463479, 20200002804114, 20200003229735, 400064244, 418163740, 432850619, 488628250, 529996768, 532344230, 534473264, 544099783, 555310530, 582483242, 592682163, 633400010, 687283937, 691797987, 878981015, 962194608, 1085100664, 1094632505, 1132032960, 1146128499, 20200000076158, 20200000092864, 20200000094966, 20200000101530, 20200000102584, 20200000103545, 20200000119865, 20200000136805, 20200000192661, 20200000213144, 20200000267909, 20200000268748, 20200000280788, 20200000363968, 20200000365351, 20200000388417, 20200000395618, 20200000400757, 20200000404655, 20200000411320, 20200000426629, 20200000427195, 20200000448596, 20200000449890, 20200000454705, 20200000462999, 20200002793748, 495321213, 507487701, 537049275, 560586497, 567471473, 678172028, 863045135, 876291194, 927859949, 972679178, 982088938, 986903497, 1083869787, 1117044600, 20200000002039, 20200000003318, 20200000016943, 20200000043415, 20200000058772, 20200000072714, 20200000080407, 20200000087425, 20200000103753, 20200000108304, 20200000116152, 20200000121894, 20200000125597, 20200000129530, 20200000130077, 20200000148467, 20200000192760, 20200000221746, 20200000250499, 20200000261183, 20200000271570, 20200000274787, 20200000282063, 20200000372805, 20200000377951, 20200000383516, 20200000407558, 20200000437076, 20200000441681, 20200000441726, 20200000448887, 20200000451970, 20200000455080, 299117180, 317469184, 496218553, 544726604, 566767002, 653453499, 675097117, 694565952, 694876905, 747977899, 868013618, 1094429166, 1117998129, 1180992004, 20200000015411, 20200000046564, 20200000089046, 20200000091721, 20200000102405, 20200000114354, 20200000121989, 20200000123510, 20200000125874, 20200000132644, 20200000147869, 20200000158320, 20200000181264, 20200000184686, 20200000191121, 20200000191494, 20200000217735, 20200000218900, 20200000249475, 20200000257645, 20200000271818, 20200000274457, 20200000381789, 20200000384132, 20200000403258, 20200000409296, 20200000429722, 20200000438743, 20200000443855, 20200000443985, 20200000446693, 20200000452718, 20200000454471, 20200000457760, 20200000461275, 20200000464197, 20200002073608, 364923860, 375610291, 406239138, 514634235, 515877714, 521640355, 525694671, 1041980048, 1059905637, 1068366903, 1081514687, 1112307917, 1181315645, 20200000005240, 20200000025635, 20200000028054, 20200000030785, 20200000042643, 20200000068795, 20200000080168, 20200000107331, 20200000128799, 20200000134823, 20200000143153, 20200000178353, 20200000200739, 20200000201977, 20200000207780, 20200000217532, 20200000221156, 20200000224779, 20200000228224, 20200000231174, 20200000247645, 20200000248797, 20200000259096, 20200000268820, 20200000275819, 20200000379708, 20200000400951, 20200000424036, 20200000435577, 20200000452641, 20200000462581, 20200002966329, 284993459, 357555848, 370422895, 377858563, 392396657, 440702623, 488568722, 498182674, 510403496, 514734550, 523635526, 531198828, 547834040, 632353530, 730873197, 840919914, 898450450, 900721327, 923441570, 942443178, 1014459049, 1028770959, 1070007906, 1084635956, 1104880869, 20200000034395, 20200000035150, 20200000074616, 20200000090984, 20200000097474, 20200000101223, 20200000107295, 20200000109228, 20200000120715, 20200000121490, 20200000127778, 20200000129872, 20200000138095, 20200000156369, 20200000157850, 20200000220685, 20200000225766, 20200000257114, 20200000260127, 20200000281564, 20200000353131, 20200000360283, 20200000364491, 20200000394970, 20200000398968, 20200000432763, 20200000434321, 20200000443176, 20200000447414, 20200000450617, 20200000459797, 20200000460527, 20200001769068, 20200001913545, 20200002195563, 20200003182784, 306811786, 396669943, 441480625, 460055278, 474461946, 757915041, 780973348, 821135385, 890740165, 894372123, 896798767, 900243182, 948805709, 971329287, 981970282, 987436926, 1043876680, 1059722265, 1116215882, 1150346397, 1154015804, 20200000002328, 20200000033446, 20200000048981, 20200000063639, 20200000072773, 20200000078630, 20200000092594, 20200000100122, 20200000131188, 20200000144722, 20200000153576, 20200000156551, 20200000165121, 20200000175565, 20200000194036, 20200000194729, 20200000205201, 20200000240087, 20200000266422, 20200000353466, 20200000360146, 20200000362821, 20200000375021, 20200000390052, 20200000404455, 20200000429202, 20200000435936, 20200000448663, 351005254, 356169777, 412823147, 449943548, 479444991, 535744485, 553211336, 609843527, 659448144, 686313581, 739707019, 803126547, 915858167, 917763923, 1038849525, 1117896933, 1127241728, 1152564998, 20200000008117, 20200000025774, 20200000035987, 20200000041178, 20200000043748, 20200000062373, 20200000076755, 20200000077461, 20200000100967, 20200000112190, 20200000112454, 20200000123980, 20200000125117, 20200000127943, 20200000128150, 20200000154490, 20200000157455, 20200000161660, 20200000201466, 20200000202256, 20200000202961, 20200000255088, 20200000258412, 20200000269722, 20200000358316, 20200000358342, 20200000370450, 20200000391494, 20200000406464, 20200000407862, 20200000416429, 20200000418629, 20200000421755, 20200000424556, 20200000448664, 20200000454122, 20200002070988, 20200002521487, 391994966, 509537638, 513850122, 562284789, 563910324, 572904333, 595761827, 620067834, 720799534, 940249281, 955243932, 966395419, 1074863853, 1076855510, 1079178967, 1118662812, 20200000028101, 20200000028766, 20200000050026, 20200000063971, 20200000087710, 20200000104474, 20200000119736, 20200000128339, 20200000132075, 20200000133590, 20200000145341, 20200000167985, 20200000168942, 20200000193628, 20200000230973, 20200000235159, 20200000237135, 20200000254109, 20200000258729, 20200000270826, 20200000362163, 20200000372858, 20200000373579, 20200000403309, 20200000414685, 20200000417542, 20200000422509, 20200000431368, 20200000447132, 20200002364391, 336353151, 337841739, 382002079, 415894286, 432667443, 479516143, 507672551, 529479190, 549132344, 553587942, 591626572, 688831692, 720063540, 724786142, 814656403, 896112560, 932236332, 958224263, 968991365, 1026311548, 1077795682, 1138360789, 1156630288, 1248807329, 20200000005062, 20200000010352, 20200000011962, 20200000027321, 20200000033264, 20200000057280, 20200000068150, 20200000116291, 20200000121767, 20200000142129, 20200000159662, 20200000163122, 20200000181229, 20200000185351, 20200000185792, 20200000193510, 20200000193726, 20200000207399, 20200000207415, 20200000240484, 20200000249744, 20200000258144, 20200000265570, 20200000267160, 20200000278746, 20200000360556, 20200000367368, 20200000367530, 20200000370551, 20200000409146, 20200000409389, 20200000421637, 20200000431974, 20200000443414, 20200000462861, 20200000463800, 20200001393988, 20200002915281, 300265786, 407781177, 483582634, 531408453, 535135084, 561048758, 570060128, 608947639, 672417831, 804766670, 863196620, 912699060, 934484238, 935224319, 984594553, 1038233279, 1067607699, 1161968147, 20200000004257, 20200000080694, 20200000126227, 20200000129165, 20200000130638, 20200000151884, 20200000158535, 20200000195288, 20200000196333, 20200000211702, 20200000212023, 20200000258242, 20200000350256, 20200000355784, 20200000356489, 20200000356996, 20200000383945, 20200000389521, 20200000405619, 20200000414490, 20200000420405, 20200000429919, 20200000440481, 20200003063329, 384216824, 395216676, 416216396, 418689935, 446207961, 464131808, 484672289, 569861156, 666978871, 675628523, 771616466, 774381416, 880047117, 917667318, 943677367, 977905957, 1033282601, 1056757544, 1157967043, 1183119353, 20200000013877, 20200000018404, 20200000022509, 20200000030711, 20200000044131, 20200000052866, 20200000074962, 20200000078392, 20200000124818, 20200000128692, 20200000162238, 20200000172720, 20200000181356, 20200000188061, 20200000202585, 20200000205098, 20200000221829, 20200000236769, 20200000251910, 20200000256538, 20200000266372, 20200000270650, 20200000273154, 20200000275047, 20200000361988, 20200000362103, 20200000362160, 20200000368184, 20200000370356, 20200000375790, 20200000379654, 20200000392713, 20200000398938, 20200000412230, 20200000432957, 20200000443522, 20200000463519, 20200002300664, 284791396, 374817863, 391234869, 498375892, 508400707, 509013878, 522350127, 576588894, 598581619, 609405853, 640480131, 848160327, 858226685, 874881636, 979413580, 1079527526, 1081847121, 1114135663, 20200000005366, 20200000036018, 20200000060394, 20200000072526, 20200000124665, 20200000131743, 20200000141123, 20200000152932, 20200000176520, 20200000181435, 20200000183993, 20200000192787, 20200000196372, 20200000200848, 20200000213992, 20200000215180, 20200000237553, 20200000251383, 20200000253366, 20200000256649, 20200000258578, 20200000271072, 20200000278455, 20200000386303, 20200000392536, 20200000397795, 20200000416504, 20200000416603, 20200000425420, 20200000434142, 20200000435643, 20200000435869, 20200000438239, 20200000443479, 20200000446181, 20200002183745, 20200002508491, 356571048, 377298193, 426382105, 429177928, 487150715, 502046597, 545715077, 547101139, 570306657, 577990521, 584435837, 698925807, 898568005, 938762460, 964555954, 991660396, 1003820457, 1224859655, 1434713683, 20200000000872, 20200000036441, 20200000052864, 20200000054485, 20200000071662, 20200000072268, 20200000086699, 20200000121439, 20200000128731, 20200000133005, 20200000138024, 20200000142031, 20200000156276, 20200000158328, 20200000162515, 20200000173510, 20200000184722, 20200000189008, 20200000190888, 20200000196439, 20200000203930, 20200000214643, 20200000243618, 20200000249523, 20200000255343, 20200000258692, 20200000281224, 20200000369139, 20200000384624, 20200000410971, 20200000414984, 20200000415110, 20200000416329, 20200000416552, 20200000417973, 20200000430270, 20200000431290, 20200000450311, 445285400, 533055109, 576328422, 623762162, 648668184, 682046579, 844014237, 930380089, 1052511603, 1054948424, 1097511960, 1100612581, 1114127463, 1502705126, 20200000018874, 20200000059472, 20200000075486, 20200000094229, 20200000112413, 20200000117774, 20200000128540, 20200000133194, 20200000133540, 20200000141368, 20200000148839, 20200000166447, 20200000170132, 20200000179626, 20200000200195, 20200000205006, 20200000206124, 20200000225380, 20200000239908, 20200000243957, 20200000247859, 20200000250568, 20200000260894, 20200000263343, 20200000274223, 20200000352595, 20200000385686, 20200000391790, 20200000420385, 20200000434280, 20200000438999, 20200000440483, 20200000443495, 20200000451391, 20200000454034, 304077942, 397553707, 400359187, 406493132, 717099050, 722217471, 731360705, 834775830, 849145835, 871125783, 919097064, 969592391, 1037420924, 1049178312, 1050017714, 1076570551, 1077697461, 1078066384, 1102814563, 1157618076, 1188118201, 1199284303, 1232099210, 1449689396, 20200000012418, 20200000017913, 20200000041050, 20200000053286, 20200000055332, 20200000058239, 20200000079550, 20200000092085, 20200000108681, 20200000109845, 20200000125173, 20200000132008, 20200000136889, 20200000147547, 20200000153705, 20200000155037, 20200000166285, 20200000180949, 20200000182562, 20200000197829, 20200000227005, 20200000227684, 20200000242650, 20200000277212, 20200000367428, 20200000382663, 20200000384611, 20200000404702, 20200000413267, 20200000432119, 20200000436387, 20200000438793, 20200000439991, 20200000447431, 20200000456480, 20200000457479, 20200000460833, 20200003097609, 313904711, 364365411, 364894352, 392933688, 426285060, 453051448, 499250722, 585161245, 625477515, 678161155, 731592936, 827448790, 887649233, 1012231677, 1032743715, 1050866909, 1052231801, 1064665530, 1086378532, 1089117838, 1119104329, 1140837908, 20200000044551, 20200000048091, 20200000092491, 20200000124669, 20200000126361, 20200000130346, 20200000136454, 20200000156104, 20200000179197, 20200000182099, 20200000184664, 20200000190388, 20200000221213, 20200000239340, 20200000239714, 20200000249936, 20200000256514, 20200000275986, 20200000283529, 20200000352879, 20200000409142, 20200000409177, 20200000411905, 20200000419279, 20200000419799, 20200000447715, 20200000448167, 20200000458637, 20200000463268, 20200003183403, 284815942, 334989259, 406668278, 528884277, 566015240, 569632660, 635252156, 700689913, 706099830, 722784048, 806077016, 867964741, 875072844, 958415876, 1058774427, 1067283885, 1156035436, 1164866427, 1499259050, 20200000003893, 20200000009361, 20200000012297, 20200000026272, 20200000057441, 20200000058692, 20200000065442, 20200000085452, 20200000087210, 20200000102316, 20200000108570, 20200000118050, 20200000139550, 20200000154048, 20200000164840, 20200000170146, 20200000189104, 20200000196525, 20200000230584, 20200000238815, 20200000256249, 20200000257358, 20200000257798, 20200000271949, 20200000272643, 20200000276310, 20200000354495, 20200000354818, 20200000361871, 20200000363239, 20200000368128, 20200000395993, 20200000399038, 20200000400409, 20200000404796, 20200000407077, 20200000407603, 20200000429902, 20200000439051, 20200000439526, 20200000440540, 20200000442513, 20200000444648, 20200000445218, 20200000454793, 20200000459049, 500544250, 506009115, 595959391, 642821482, 664973122, 692412579, 739758468, 740146917, 792595594, 868101848, 946903436, 1005050125, 1073500846, 1132537738, 20200000003459, 20200000023269, 20200000025020, 20200000047968, 20200000076885, 20200000098136, 20200000100677, 20200000123743, 20200000124276, 20200000144147, 20200000149503, 20200000164861, 20200000171012, 20200000195639, 20200000205834, 20200000226549, 20200000228709, 20200000247007, 20200000255709, 20200000266419, 20200000273404, 20200000277163, 20200000350305, 20200000351550, 20200000367362, 20200000370952, 20200000376159, 20200000387838, 20200000410338, 20200000425959, 20200000446001, 20200000446815, 20200000457505, 20200001832110, 375603179, 419020937, 437352931, 545861112, 581463445, 582805911, 604921715, 692035811, 842525943, 879569662, 884206090, 926570255, 942926004, 953640451, 978659937, 980151457, 983463316, 1010961376, 1037959338, 1072251692, 1103094508, 20200000002368, 20200000013004, 20200000013992, 20200000025348, 20200000045586, 20200000050422, 20200000072975, 20200000079871, 20200000085610, 20200000086919, 20200000090233, 20200000104557, 20200000111656, 20200000148286, 20200000187618, 20200000191240, 20200000203255, 20200000206742, 20200000211381, 20200000222426, 20200000241008, 20200000241269, 20200000242398, 20200000249055, 20200000250580, 20200000257837, 20200000264169, 20200000282039, 20200000350543, 20200000360630, 20200000372447, 20200000376057, 20200000415165, 20200000415915, 20200000419285, 20200000427561, 20200000441035, 20200000445339, 20200000455128, 20200002166099, 20200002915348, 367003839, 376291994, 383718755, 467577200, 472885640, 525956148, 566323324, 623789705, 733788136, 796843000, 922744882, 927523832, 937685528, 961100433, 984536464, 1040172229, 1098307964, 1147587082, 20200000024122, 20200000024244, 20200000031730, 20200000079818, 20200000079998, 20200000098703, 20200000130502, 20200000148875, 20200000158062, 20200000171294, 20200000175411, 20200000204403, 20200000217084, 20200000230487, 20200000259341, 20200000263959, 20200000268828, 20200000276757, 20200000350774, 20200000357181, 20200000396557, 20200000434883, 20200000436082, 20200000452956, 20200000461327, 20200000462667, 20200002348596, 20200002473258, 479832296, 498944221, 506287856, 562413829, 587360671, 659336346, 713448825, 810933446, 813671555, 830544402, 878625143, 891755554, 893372761, 904242072, 911800950, 928841244, 949870726, 992309647, 996887750, 1061768547, 1113603774, 1192113927, 20200000001100, 20200000015700, 20200000021782, 20200000029148, 20200000042289, 20200000077848, 20200000098733, 20200000136580, 20200000144557, 20200000152969, 20200000163980, 20200000220241, 20200000223766, 20200000232971, 20200000238433, 20200000270193, 20200000279979, 20200000351038, 20200000353315, 20200000355285, 20200000368415, 20200000398463, 20200000425974, 20200000428602, 20200000438689, 20200000440441, 20200000442640, 20200000451057, 20200000451832, 20200000458056, 20200000461308, 20200000468612, 20200002083077, 20200002706667, 281656475, 288429040, 292738169, 306310789, 585801479, 640709006, 696639740, 753424887, 848264096, 862360479, 875242534, 930691472, 933034219, 959937622, 1067571483, 1115681877, 1129176750, 1181261241, 1182321057, 1232518901, 20200000039983, 20200000043642, 20200000049378, 20200000114723, 20200000124057, 20200000134717, 20200000137900, 20200000156772, 20200000177643, 20200000181431, 20200000196901, 20200000205097, 20200000237568, 20200000238295, 20200000245574, 20200000265667, 20200000276766, 20200000277810, 20200000355599, 20200000355953, 20200000362169, 20200000370871, 20200000374036, 20200000409812, 20200000419847, 20200000419911, 20200000420898, 20200000433586, 20200000445912, 20200000451654, 20200000456890, 20200000460726, 20200002890056, 293614976, 312220102, 358273498, 393149734, 422829104, 484546703, 505401716, 571865683, 577628510, 626583252, 670213870, 692717444, 919087726, 994102781, 1099445683, 1182580495, 20200000003039, 20200000011828, 20200000016217, 20200000051541, 20200000063732, 20200000078730, 20200000146687, 20200000150358, 20200000152607, 20200000156765, 20200000193238, 20200000222133, 20200000222710, 20200000236617, 20200000247001, 20200000251792, 20200000274155, 20200000359644, 20200000360479, 20200000362991, 20200000363902, 20200000370545, 20200000372116, 20200000372429, 20200000379402, 20200000381853, 20200000403870, 20200000412229, 20200000416955, 20200000426607, 20200000449340, 20200000455414, 20200000456382, 20200000459247, 20200000461952, 20200002750300, 20200003225435, 324923351, 358126452, 439628153, 500098096, 513265579, 538838157, 591325376, 656176278, 666245333, 673354210, 678861146, 720041510, 876656488, 897995376, 912155301, 990066072, 992152964, 1010801519, 1088651004, 1093108529, 1114807871, 1151544821, 1164930261, 1476527928, 20200000069766, 20200000071610, 20200000073068, 20200000078169, 20200000083903, 20200000084007, 20200000174824, 20200000186047, 20200000199724, 20200000202116, 20200000217481, 20200000221003, 20200000224794, 20200000230932, 20200000247180, 20200000255571, 20200000266430, 20200000279410, 20200000354780, 20200000354828, 20200000362663, 20200000382974, 20200000388638, 20200000390038, 20200000401496, 20200000404458, 20200000417040, 20200000418661, 20200000432382, 20200000440850, 20200000441666, 20200000443853, 20200000444077, 288927565, 348867501, 364481787, 364694031, 388463936, 475772902, 477537958, 490952152, 541448708, 561664830, 566407600, 571639319, 581462234, 747655892, 925209077, 931840392, 935754064, 942553473, 1078850198, 1100991397, 1106014973, 1108417718, 1176547358, 1181733278, 1486923376, 20200000017094, 20200000075469, 20200000090577, 20200000102541, 20200000118488, 20200000175582, 20200000189849, 20200000190722, 20200000191660, 20200000210043, 20200000227380, 20200000242846, 20200000243805, 20200000252443, 20200000359431, 20200000378095, 20200000391571, 20200000394478, 20200000417127, 20200000425052, 20200000437082, 20200000437220, 20200000444640, 20200000455389, 527588389, 658439903, 754105884, 796333129, 859243872, 919658560, 1042814349, 1062421758, 1066348440, 1088293809, 1104933884, 1128691441, 1129854785, 20200000006174, 20200000020936, 20200000042903, 20200000053294, 20200000072118, 20200000077858, 20200000079973, 20200000092454, 20200000161636, 20200000164922, 20200000169048, 20200000178301, 20200000179934, 20200000196355, 20200000208144, 20200000210578, 20200000211925, 20200000218084, 20200000232988, 20200000233507, 20200000237207, 20200000243665, 20200000249852, 20200000252898, 20200000258036, 20200000259765, 20200000267969, 20200000277012, 20200000352507, 20200000371220, 20200000392412, 20200000397188, 20200000397850, 20200000409178, 20200000426596, 20200000427697, 20200000440552, 20200000444477, 20200000448541, 20200000454658, 20200000460551, 20200000461887, 20200002145015, 20200002438631, 438210640, 578909807, 642727743, 794748149, 911695438, 917948333, 983733111, 986125962, 1143445119, 1180482499, 20200000001733, 20200000006080, 20200000022274, 20200000024399, 20200000039491, 20200000054148, 20200000055600, 20200000114740, 20200000121151, 20200000124430, 20200000125876, 20200000142358, 20200000178885, 20200000185745, 20200000215455, 20200000223644, 20200000223915, 20200000227059, 20200000256625, 20200000282590, 20200000352530, 20200000358079, 20200000366036, 20200000375199, 20200000397532, 20200000408528, 20200000429788, 20200000432736, 20200000451647, 20200000453415, 20200000457087, 20200000460355, 20200000461365, 20200000461856, 20200001478379, 346015341, 354015291, 382011064, 383512724, 438351943, 474710598, 479662730, 512643034, 515953991, 593418681, 659283309, 716913565, 763692274, 803857990, 893834568, 926078360, 965803630, 972558973, 1010803520, 1060085514, 1062583591, 1068417354, 20200000016216, 20200000040918, 20200000058969, 20200000061298, 20200000106763, 20200000115851, 20200000163648, 20200000180841, 20200000212200, 20200000217193, 20200000225596, 20200000251926, 20200000252513, 20200000256050, 20200000271020, 20200000271708, 20200000355178, 20200000357779, 20200000359203, 20200000374524, 20200000385368, 20200000400072, 20200000403414, 20200000420277, 20200000422470, 20200000437268, 20200000439023, 20200000443538, 20200000461707, 288113403, 301046057, 345448092, 454316134, 474940131, 475976577, 482337530, 630864653, 666185138, 697922531, 763423846, 914937512, 926017108, 966671775, 993235298, 1116025351, 1173663630, 20200000021501, 20200000068743, 20200000081412, 20200000085446, 20200000089546, 20200000102898, 20200000104075, 20200000114021, 20200000150079, 20200000154434, 20200000166733, 20200000169956, 20200000180495, 20200000193068, 20200000209735, 20200000214244, 20200000217363, 20200000218977, 20200000248874, 20200000253131, 20200000253460, 20200000264143, 20200000267563, 20200000282700, 20200000360892, 20200000368264, 20200000371391, 20200000385815, 20200000396898, 20200000410574, 20200000410882, 20200000437003, 20200000441292, 20200000448512, 20200000461610, 20200000462979, 20200002429373, 370488535, 383790206, 432194840, 489479624, 526783584, 562558324, 586683407, 657500465, 693256675, 721943975, 802700319, 862872087, 910570079, 933279284, 950593464, 955705796, 971304016, 1071976327, 1082803056, 1151195880, 1165320455, 20200000035357, 20200000108391, 20200000119681, 20200000120537, 20200000126475, 20200000154002, 20200000158549, 20200000176345, 20200000176576, 20200000215076, 20200000218683, 20200000240751, 20200000247981, 20200000353938, 20200000361103, 20200000363950, 20200000371775, 20200000381491, 20200000385181, 20200000394060, 20200000408948, 20200000414108, 20200000435425, 333578095, 364518740, 392765650, 404769720, 422671573, 426811787, 468670618, 469343097, 529096824, 577586159, 585193266, 651510680, 656951157, 730081072, 730712409, 872615882, 885792725, 982175678, 1023368535, 1049508363, 1080634585, 1138642849, 1156121311, 20200000055950, 20200000101341, 20200000114231, 20200000134092, 20200000146265, 20200000161818, 20200000168146, 20200000204619, 20200000208675, 20200000232425, 20200000247982, 20200000256187, 20200000265561, 20200000268990, 20200000270142, 20200000272171, 20200000273488, 20200000279890, 20200000352440, 20200000352768, 20200000353847, 20200000370686, 20200000383757, 20200000391774, 20200000412743, 20200000419481, 20200000433367, 20200000436195, 20200000439994, 20200000443254, 20200000446724, 20200000455478, 20200000463266, 20200002207451, 351331194, 393231526, 405506640, 485126024, 510305695, 590250573, 639785198, 666790726, 757811340, 882119723, 893677228, 905488045, 918820076, 970021572, 1060704812, 1156258040, 1178082935, 20200000005662, 20200000019795, 20200000030855, 20200000046599, 20200000060975, 20200000071347, 20200000095819, 20200000107740, 20200000113795, 20200000116054, 20200000117678, 20200000118326, 20200000144163, 20200000144868, 20200000165122, 20200000190519, 20200000198216, 20200000207406, 20200000223922, 20200000225937, 20200000234613, 20200000255245, 20200000264652, 20200000265740, 20200000282305, 20200000353367, 20200000368215, 20200000377423, 20200000388452, 20200000393469, 20200000402353, 20200000417324, 20200000424038, 20200000425310, 20200000436351, 20200000440496, 20200000444878, 20200000446310, 20200000451496, 20200000453949, 20200000455546, 20200000455761, 20200000463113, 20200003095189, 404167616, 422142590, 456697773, 496176795, 503143550, 517642765, 526422908, 548325168, 561577469, 634612662, 909911026, 911793120, 964436963, 997878440, 1057925633, 1068546096, 1070075833, 1104890554, 1156447964, 1175277892, 20200000006234, 20200000021100, 20200000022912, 20200000033289, 20200000049621, 20200000054868, 20200000065514, 20200000068896, 20200000082966, 20200000096956, 20200000102215, 20200000111758, 20200000115378, 20200000127670, 20200000139975, 20200000162921, 20200000198421, 20200000225309, 20200000241452, 20200000243882, 20200000249346, 20200000267939, 20200000276600, 20200000276901, 20200000352647, 20200000353233, 20200000368557, 20200000399050, 20200000410770, 20200000422253, 20200000428754, 20200000428765, 20200000435441, 20200000439394, 20200000453045, 20200000458622, 20200000462695, 20200000613940, 20200002292519, 20200003182603, 281747159, 294892942, 337552456, 363634698, 413126268, 459850726, 504520620, 598603610, 623355278, 690541689, 718120892, 845139162, 932417099, 1003588804, 1020481008, 1021566244, 1028464453, 1075872386, 1148153611, 1252933794, 20200000068772, 20200000077197, 20200000096008, 20200000109505, 20200000112209, 20200000140146, 20200000144179, 20200000190955, 20200000193363, 20200000194048, 20200000207735, 20200000233354, 20200000236377, 20200000238409, 20200000244072, 20200000245089, 20200000253683, 20200000255378, 20200000255442, 20200000279261, 20200000350513, 20200000351454, 20200000371327, 20200000387515, 20200000445931, 20200000452906, 20200003119020, 358121343, 369626098, 404452499, 407870968, 469933147, 531204555, 535076358, 543545550, 596677177, 934596429, 966897134, 1032460746, 1037742160, 1058777280, 1065452419, 1085248903, 1121573306, 1178324336, 1180562524, 1218666033, 1423979633, 20200000003775, 20200000060555, 20200000067771, 20200000069809, 20200000077773, 20200000124074, 20200000134653, 20200000139559, 20200000183062, 20200000211883, 20200000257250, 20200000258785, 20200000260208, 20200000275016, 20200000365717, 20200000383197, 20200000385075, 20200000407116, 20200000410156, 20200000417440, 20200000421252, 20200000428062, 20200000435605, 20200000442826, 20200000446678, 20200000447339, 20200000449847, 20200000451070, 20200000452358, 20200000452419, 20200000461883, 20200000462247, 20200000464276, 20200002243035, 320204481, 331705067, 364942594, 441760154, 447553564, 472937654, 513846004, 673017568, 715822238, 724816878, 768600096, 1014842228, 1021239565, 1031843950, 1055502792, 1092434028, 1122444037, 1138240741, 1151054926, 1485524090, 20200000023795, 20200000040896, 20200000062389, 20200000071587, 20200000073711, 20200000076106, 20200000085030, 20200000092656, 20200000098166, 20200000105764, 20200000108911, 20200000110500, 20200000117984, 20200000119717, 20200000128038, 20200000138231, 20200000155134, 20200000173629, 20200000191280, 20200000218372, 20200000230056, 20200000233802, 20200000236654, 20200000240292, 20200000265034, 20200000357651, 20200000361105, 20200000370811, 20200000401954, 20200000410382, 20200000412503, 20200000419654, 20200000420318, 20200000422858, 20200000424202, 20200000437942, 20200000441251, 20200000442420, 20200000449222, 20200000451158, 20200000454876, 20200001627737, 20200001735515, 20200003074625, 377704407, 385168328, 507874739, 509448307, 524685220, 530232671, 535500008, 653508448, 700970012, 749083919, 972622980, 985169760, 986113624, 20200000014572, 20200000019386, 20200000030862, 20200000038996, 20200000053343, 20200000058686, 20200000061962, 20200000072429, 20200000099498, 20200000115616, 20200000143151, 20200000143252, 20200000159019, 20200000159272, 20200000173365, 20200000184662, 20200000212208, 20200000218136, 20200000354124, 20200000354360, 20200000366573, 20200000371462, 20200000371768, 20200000374346, 20200000382567, 20200000398046, 20200000402220, 20200000404755, 20200000407264, 20200000409351, 20200000413375, 20200000428932, 20200000437518, 20200000446428, 20200000456930, 389781154, 409794756, 436851416, 473681736, 534231417, 586447913, 635325621, 672979566, 674184945, 693138374, 796383629, 885788690, 899087374, 1084930849, 1174204073, 20200000024782, 20200000031112, 20200000033725, 20200000068685, 20200000070455, 20200000076848, 20200000078243, 20200000087135, 20200000115013, 20200000140691, 20200000140775, 20200000142076, 20200000163043, 20200000170793, 20200000180141, 20200000218138, 20200000223248, 20200000225092, 20200000257141, 20200000276027, 20200000277770, 20200000352513, 20200000353090, 20200000415717, 20200000418225, 20200000420656, 20200000431157, 20200000434343, 20200000438163, 20200000441762, 20200000447473, 384312640, 404182204, 495680460, 574641836, 578448682, 584927317, 591117706, 593847174, 625263720, 686542963, 742136923, 883393043, 897062509, 961281779, 966030326, 981633844, 1001059684, 1118850348, 1123300756, 1124365642, 1146465836, 1404854993, 20200000029818, 20200000054513, 20200000057980, 20200000068346, 20200000083761, 20200000090275, 20200000169574, 20200000176031, 20200000176317, 20200000184604, 20200000185492, 20200000199696, 20200000207491, 20200000232844, 20200000250531, 20200000254419, 20200000258842, 20200000262593, 20200000277354, 20200000282045, 20200000362130, 20200000365751, 20200000378491, 20200000380243, 20200000401129, 20200000438032, 20200001761487, 20200002313992, 20200002368807, 369176364, 405239907, 504001448, 543577420, 555916407, 561952062, 654815966, 660188693, 714127557, 719401490, 819197891, 947134111, 1042987645, 1070489887, 1156787368, 1186629823, 20200000001596, 20200000011831, 20200000040008, 20200000045753, 20200000062623, 20200000067157, 20200000091466, 20200000094015, 20200000128747, 20200000186682, 20200000193781, 20200000211857, 20200000222471, 20200000248190, 20200000281376, 20200000372550, 20200000379665, 20200000391891, 20200000400010, 20200000403568, 20200000407279, 20200000412117, 20200000415755, 20200000434120, 20200000438227, 20200000446544, 20200000447070, 20200000447348, 20200002003190, 20200002421751, 404154726, 410896080, 420636551, 436114747, 444947784, 478829300, 479278688, 576764076, 596684220, 642831690, 672924134, 698255242, 933471941, 942990867, 948607270, 1021094389, 1113503715, 1229581261, 1236825910, 20200000043186, 20200000093160, 20200000107088, 20200000128850, 20200000133096, 20200000144967, 20200000163694, 20200000172266, 20200000216251, 20200000223496, 20200000259833, 20200000272748, 20200000358020, 20200000367612, 20200000403536, 20200000417670, 20200000426334, 20200000435530, 20200000458454, 20200000463438, 20200001458938, 20200001765595, 284979163, 317809458, 355240912, 427769832, 454638411, 505046678, 680194574, 863003811, 1000876192, 1090659100, 1151747316, 1179637054, 20200000001219, 20200000004686, 20200000006449, 20200000022776, 20200000035694, 20200000065893, 20200000134002, 20200000134851, 20200000144733, 20200000176450, 20200000211053, 20200000211510, 20200000221537, 20200000226847, 20200000247383, 20200000248828, 20200000250211, 20200000273141, 20200000282482, 20200000353691, 20200000354437, 20200000356960, 20200000370116, 20200000399655, 20200000401613, 20200000404798, 20200000414679, 20200000422664, 20200000435474, 20200000439581, 20200000444605, 20200000444691, 20200000453116, 20200000455599, 20200000458117, 20200001680612, 20200003232006, 430845597, 536067618, 687550747, 728293409, 735679002, 735945527, 767539394, 875267105, 893673936, 970417047, 1073476012, 1136835359, 1145465310, 1460010365, 20200000017360, 20200000020925, 20200000028574, 20200000053651, 20200000083590, 20200000084464, 20200000123960, 20200000131672, 20200000142174, 20200000169118, 20200000198444, 20200000206296, 20200000246512, 20200000271577, 20200000278608, 20200000354018, 20200000356636, 20200000363313, 20200000390058, 20200000397880, 20200000402827, 20200000413806, 20200000443835, 20200000446522, 20200000452653, 20200000458899, 20200000459321, 20200002176171, 20200002544882, 320279293, 391285862, 528390486, 535378448, 599516631, 655895351, 667461862, 768469908, 894087515, 944808252, 1068512694, 1175953961, 20200000006353, 20200000007561, 20200000013000, 20200000013928, 20200000017827, 20200000024856, 20200000029282, 20200000043613, 20200000071596, 20200000092487, 20200000110011, 20200000140829, 20200000141928, 20200000195564, 20200000220967, 20200000233149, 20200000249032, 20200000261563, 20200000356035, 20200000366896, 20200000395710, 20200000396748, 20200000402663, 20200000403592, 20200000408420, 20200000416323, 20200000418206, 20200000450340, 20200000452632, 20200000461873, 20200000464040, 284882215, 368243228, 409817284, 515094775, 553834731, 622613679, 656124384, 875882934, 941380906, 955856059, 957732285, 972297697, 1002064879, 1072950932, 20200000003697, 20200000005888, 20200000010003, 20200000027114, 20200000076338, 20200000104353, 20200000106898, 20200000114928, 20200000130886, 20200000133163, 20200000159119, 20200000171869, 20200000179517, 20200000200888, 20200000223093, 20200000226795, 20200000256877, 20200000270225, 20200000360374, 20200000360845, 20200000387904, 20200000395016, 20200000397970, 20200000402096, 20200000412916, 20200000413058, 20200000419761, 20200000427903, 20200000446055, 20200003134941, 336769551, 349160522, 381884632, 384855427, 439438619, 474011277, 505155771, 533724513, 613398383, 733707729, 823703847, 880101933, 898724353, 951627425, 1002426301, 1132214105, 1135451838, 1184228834, 20200000076828, 20200000101143, 20200000106774, 20200000109562, 20200000110674, 20200000122807, 20200000136827, 20200000139014, 20200000166093, 20200000167594, 20200000173440, 20200000179903, 20200000185283, 20200000189548, 20200000203374, 20200000209055, 20200000214909, 20200000219512, 20200000226090, 20200000236661, 20200000257420, 20200000356511, 20200000363442, 20200000380556, 20200000381419, 20200000409041, 20200000413266, 20200000413785, 20200000417022, 20200000417942, 20200000421128, 20200000422965, 20200000429936, 20200000433362, 20200000434287, 20200000448202, 20200000452484, 20200000457470, 289224933, 384777965, 434396747, 466966101, 492571492, 856474365, 878118022, 1065249424, 1069691018, 1120547622, 1141742842, 1183628728, 20200000017189, 20200000017291, 20200000022489, 20200000046122, 20200000055460, 20200000075878, 20200000077240, 20200000084367, 20200000085823, 20200000091939, 20200000115439, 20200000127242, 20200000130894, 20200000131993, 20200000160244, 20200000160687, 20200000195578, 20200000206227, 20200000222811, 20200000223539, 20200000243576, 20200000253297, 20200000258630, 20200000261001, 20200000268036, 20200000277602, 20200000280520, 20200000352017, 20200000363662, 20200000365197, 20200000366292, 20200000374795, 20200000382149, 20200000406310, 20200000416062, 20200000429082, 20200000434568, 20200000441659, 20200000442737, 20200000445777, 20200000451910, 20200000457711, 20200000458209, 20200002093663, 301387274, 459747886, 578250301, 594261405, 613279493, 731852306, 899003026, 920133658, 938207958, 1000706687, 1031121030, 1034238724, 1062006344, 1153515989, 20200000011327, 20200000020663, 20200000059103, 20200000074975, 20200000107114, 20200000122314, 20200000125935, 20200000135299, 20200000138149, 20200000176130, 20200000180798, 20200000184764, 20200000184827, 20200000223654, 20200000223979, 20200000229145, 20200000261781, 20200000264099, 20200000364293, 20200000402275, 20200000412606, 20200000415166, 20200000420142, 20200000422423, 20200000429591, 20200000433594, 20200000435154, 20200000438622, 20200000441934, 20200000452677, 20200001708511, 20200002449835, 363448251, 396477497, 485246824, 503143306, 575119311, 838365896, 904278510, 905729701, 983918347, 991339513, 997392079, 1031214682, 1058370532, 1075901015, 1136188121, 1168645184, 1183121077, 20200000013366, 20200000034607, 20200000043856, 20200000069702, 20200000093713, 20200000105762, 20200000111350, 20200000115529, 20200000158898, 20200000168707, 20200000180433, 20200000187285, 20200000212591, 20200000214579, 20200000221065, 20200000227647, 20200000250207, 20200000263205, 20200000264861, 20200000272790, 20200000352275, 20200000361708, 20200000382227, 20200000385260, 20200000421924, 20200000434440, 20200000439938, 20200000440439, 20200000446000, 20200000463264, 325664533, 348793475, 533079551, 573916046, 650377962, 698070860, 783214985, 824318267, 913235482, 922486324, 961618336, 982119760, 20200000032909, 20200000069460, 20200000078173, 20200000081482, 20200000087464, 20200000098195, 20200000098960, 20200000130032, 20200000151700, 20200000163993, 20200000178644, 20200000246299, 20200000255905, 20200000264905, 20200000266364, 20200000357697, 20200000377426, 20200000385986, 20200000395439, 20200000401810, 20200000417986, 20200000419894, 20200000422516, 20200000427112, 20200000434797, 20200000435739, 20200000445313, 20200000447918, 346877580, 401643317, 407838198, 426415634, 427540962, 483071025, 487275609, 492569921, 557155750, 558117780, 577153082, 719008595, 923816979, 937718942, 1017367213, 1041808803, 1067589627, 1083229791, 1131251130, 1161068344, 1166596707, 20200000000002, 20200000031526, 20200000054902, 20200000083651, 20200000114646, 20200000124151, 20200000132509, 20200000139907, 20200000148885, 20200000163495, 20200000190270, 20200000196236, 20200000197996, 20200000203661, 20200000245833, 20200000247269, 20200000248928, 20200000270148, 20200000281454, 20200000355392, 20200000361615, 20200000362625, 20200000366171, 20200000379490, 20200000385727, 20200000399741, 20200000400979, 20200000403792, 20200000419218, 20200000423050, 20200000429498, 20200000432280, 20200000439891, 20200000446111, 20200000457063, 20200000620417, 20200000974978, 20200001535297, 20200001702939, 364859644, 557576867, 608080786, 639930688, 826692859, 908653938, 1022409608, 1023648594, 20200000002671, 20200000027800, 20200000044127, 20200000078346, 20200000078754, 20200000103590, 20200000118860, 20200000119946, 20200000141539, 20200000151854, 20200000152820, 20200000193333, 20200000201780, 20200000201972, 20200000255202, 20200000255839, 20200000258417, 20200000352646, 20200000359409, 20200000363829, 20200000374216, 20200000379230, 20200000388537, 20200000394791, 20200000405883, 20200000407113, 20200000422707, 20200000423314, 20200000426902, 20200000428215, 20200000431775, 20200000441799, 20200000446686, 333235419, 362484474, 460177396, 572534490, 574123647, 598945891, 776286527, 879654575, 891280237, 993102255, 1079464948, 1445394101, 20200000025566, 20200000041465, 20200000052181, 20200000062035, 20200000063773, 20200000066245, 20200000078347, 20200000078634, 20200000085852, 20200000106126, 20200000119657, 20200000127334, 20200000139088, 20200000154779, 20200000170348, 20200000207768, 20200000247272, 20200000278192, 20200000354701, 20200000358596, 20200000393317, 20200000430146, 20200000430617, 20200000431317, 20200000457069, 20200002302105, 20200003044279, 326885152, 370462252, 373767768, 399763903, 400666114, 461402734, 583222866, 583728799, 623858130, 650607475, 747747937, 920007273, 934138990, 941617814, 959883039, 986110430, 1014565551, 1018749601, 1090653945, 1181613202, 20200000001797, 20200000051175, 20200000075554, 20200000094341, 20200000103467, 20200000109908, 20200000123344, 20200000157939, 20200000187946, 20200000199080, 20200000222206, 20200000255633, 20200000258745, 20200000260983, 20200000267332, 20200000267617, 20200000352656, 20200000356679, 20200000361016, 20200000371460, 20200000372753, 20200000374518, 20200000385226, 20200000412412, 20200000412648, 20200000413378, 20200000413973, 20200000428268, 20200000437313, 20200000441190, 20200000449557, 20200000451960, 20200000453394, 20200000457722, 20200000460730, 20200000463511, 20200001610324, 20200001688879, 20200002148355, 20200002692031, 20200002840712, 297654251, 365788937, 365912485, 382028147, 399945986, 534282109, 610295719, 611045270, 836464625, 839618264, 915375908, 919141988, 945490355, 950812012, 978866413, 998865645, 20200000001071, 20200000008065, 20200000016775, 20200000025909, 20200000034275, 20200000045680, 20200000079222, 20200000086323, 20200000118038, 20200000127244, 20200000176866, 20200000180087, 20200000182650, 20200000190770, 20200000191633, 20200000209235, 20200000247378, 20200000274081, 20200000278386, 20200000281367, 20200000354982, 20200000366903, 20200000376111, 20200000385783, 20200000406863, 20200000408398, 20200000412314, 20200000413981, 20200000419250, 20200000436446, 20200000442720, 20200000445141, 20200000461311, 20200001511491, 312528655, 374367975, 416345444, 422689480, 437655327, 637863666, 691263113, 757871909, 868692227, 910062893, 957636824, 968604229, 1017064541, 1030422375, 1035254346, 1040670798, 1113619083, 1180167123, 20200000074976, 20200000075195, 20200000079922, 20200000080214, 20200000090037, 20200000147610, 20200000188163, 20200000201001, 20200000209945, 20200000212999, 20200000213583, 20200000226723, 20200000246870, 20200000400865, 20200000424407, 20200000424513, 20200000442638, 20200000443844, 20200000448852, 20200001976090, 20200002031492, 406548272, 424575621, 481657423, 483165745, 522953445, 661113460, 680581294, 898838405, 944877545, 950593959, 952313720, 1015049223, 1040083067, 1057510303, 1148421905, 1157555235, 1164761834, 1186830317, 1498947191, 20200000001579, 20200000025057, 20200000035084, 20200000056696, 20200000058534, 20200000066782, 20200000068038, 20200000071815, 20200000089036, 20200000123568, 20200000132015, 20200000136464, 20200000143257, 20200000143915, 20200000178595, 20200000180596, 20200000182424, 20200000192125, 20200000193113, 20200000195580, 20200000198660, 20200000202874, 20200000213429, 20200000216257, 20200000239872, 20200000257259, 20200000270096, 20200000271424, 20200000278415, 20200000279865, 20200000356105, 20200000356377, 20200000357982, 20200000366796, 20200000379853, 20200000389506, 20200000402434, 20200000404165, 20200000409176, 20200000439479, 20200000446126, 20200000457595, 20200000458606, 20200000462830, 288760989, 288900569, 307979160, 364500115, 375836908, 384134949, 388885957, 398602573, 415899024, 441430041, 449820506, 493983728, 497361777, 529433904, 547974555, 586669403, 712391758, 725956878, 769737983, 790261068, 896842572, 1010444527, 1119633367, 1155904485, 1158901506, 20200000017610, 20200000048976, 20200000059094, 20200000065682, 20200000066339, 20200000067969, 20200000134814, 20200000150593, 20200000155485, 20200000162812, 20200000165504, 20200000169830, 20200000173418, 20200000193222, 20200000202592, 20200000231890, 20200000240328, 20200000242418, 20200000253321, 20200000354660, 20200000356260, 20200000356596, 20200000366157, 20200000372127, 20200000381577, 20200000392864, 20200000402415, 20200000417898, 20200000422063, 20200000425902, 20200000437444, 20200000447258, 20200000458675, 20200000859459, 20200001390881, 20200002830320, 544237538, 550089956, 575154654, 610003290, 696701152, 779157948, 779561331, 860620713, 877638937, 887640616, 907210971, 917176209, 976548062, 1019005722, 1048988466, 20200000025397, 20200000054402, 20200000086827, 20200000108721, 20200000122466, 20200000128336, 20200000132505, 20200000158715, 20200000176585, 20200000200631, 20200000219170, 20200000225445, 20200000248297, 20200000271206, 20200000363863, 20200000363995, 20200000364588, 20200000370223, 20200000371316, 20200000375146, 20200000378939, 20200000391318, 20200000391474, 20200000393751, 20200000396746, 20200000398851, 20200000402474, 20200000410868, 20200000418525, 20200000419365, 20200000428930, 20200000431586, 20200000444590, 20200000446008, 20200000451138, 20200000451480, 20200000451817, 20200000452187, 20200000456027, 20200000462879, 20200002351433, 20200002639532, 362949845, 363201632, 392762847, 530621395, 545626598, 556884588, 565911819, 581173415, 659276756, 820326185, 883539873, 978184284, 1023217192, 1033123014, 1033581397, 1034206836, 1047008947, 1050069506, 1121638579, 1129192831, 1148048602, 1156137155, 1162099512, 1176381440, 1185774325, 1195520158, 1442697588, 1499952623, 20200000007958, 20200000059311, 20200000064180, 20200000077543, 20200000097825, 20200000129390, 20200000141500, 20200000150799, 20200000167449, 20200000174312, 20200000174347, 20200000182063, 20200000195096, 20200000213816, 20200000219395, 20200000226164, 20200000239328, 20200000247412, 20200000360637, 20200000361377, 20200000369742, 20200000392014, 20200000393797, 20200000413691, 20200000416623, 20200000417892, 20200000426508, 20200000427248, 20200000432153, 20200000456612, 20200000461454, 306169895, 378368562, 400248511, 415917102, 418987775, 430152294, 469359021, 489597379, 512939461, 535886823, 571625654, 703540206, 708388097, 716977895, 808431586, 933760266, 960436333, 998471673, 1023200944, 1030805441, 1036517151, 1058959277, 1145317925, 1498416097, 20200000003074, 20200000011188, 20200000028770, 20200000032920, 20200000039114, 20200000048805, 20200000049372, 20200000076245, 20200000097945, 20200000139496, 20200000143624, 20200000158563, 20200000166280, 20200000180169, 20200000187551, 20200000243415, 20200000253701, 20200000254523, 20200000255867, 20200000258445, 20200000261665, 20200000274355, 20200000361721, 20200000365075, 20200000368177, 20200000374457, 20200000376827, 20200000384760, 20200000387563, 20200000395469, 20200000414855, 20200000421889, 20200000427334, 20200000436355, 20200000448869, 20200000449651, 20200000462641, 20200000462666, 20200000643829, 20200002994724, 317911945, 364017607, 522408559, 523067769, 580765736, 591622040, 635966718, 668692393, 703317048, 841643980, 889581369, 935623257, 1048260999, 1078810032, 1093914796, 1109924240, 1145344860, 1498448097, 20200000006927, 20200000055546, 20200000062616, 20200000064167, 20200000073916, 20200000092915, 20200000101423, 20200000158135, 20200000171628, 20200000204067, 20200000205920, 20200000223300, 20200000251900, 20200000255462, 20200000279574, 20200000358260, 20200000363074, 20200000385552, 20200000399642, 20200000427363, 20200000433597, 20200000451913, 20200000453700, 20200001730708, 20200001846670, 388481264, 388614277, 421511821, 442689429, 448440398, 485362757, 491539145, 541913955, 625334537, 731645633, 749046891, 781507616, 875772601, 922663098, 925490525, 935216956, 973350682, 1001911970, 1043318628, 1080114119, 1168511834, 1169979118, 20200000001086, 20200000022542, 20200000036283, 20200000055594, 20200000106669, 20200000136997, 20200000176816, 20200000181489, 20200000184633, 20200000202861, 20200000202995, 20200000210753, 20200000218750, 20200000227578, 20200000242413, 20200000276887, 20200000451003, 20200000452071, 20200002024333, 20200002280211, 476312888, 537936449, 549872444, 651978174, 749094938, 799501867, 806085783, 816158408, 850057092, 871694174, 880971164, 917683741, 1188321787, 1229471485, 20200000037146, 20200000039232, 20200000053678, 20200000056504, 20200000061436, 20200000065743, 20200000080595, 20200000099387, 20200000106649, 20200000107471, 20200000116596, 20200000132373, 20200000141931, 20200000159953, 20200000178154, 20200000179079, 20200000185652, 20200000187640, 20200000190422, 20200000198275, 20200000245051, 20200000257528, 20200000271593, 20200000273051, 20200000354689, 20200000357303, 20200000358982, 20200000362788, 20200000368398, 20200000369155, 20200000386646, 20200000390903, 20200000404720, 20200000416946, 20200000424477, 20200000424578, 20200000425315, 20200000430276, 20200000443058, 20200000446777, 20200000447397, 20200000450532, 20200002964317, 334876403, 412099246, 457351394, 503529453, 605569663, 608901634, 692669501, 706000131, 778300400, 788393361, 804360921, 808351987, 814434907, 830250161, 899441948, 994407389, 1024617728, 1046593064, 1057532723, 1081079080, 1084805156, 1088545329, 1106831630, 1145829062, 1148447147, 1182289149, 20200000003562, 20200000008849, 20200000015217, 20200000027608, 20200000036695, 20200000036985, 20200000069242, 20200000091756, 20200000104018, 20200000111915, 20200000130212, 20200000136407, 20200000154174, 20200000162476, 20200000165987, 20200000174310, 20200000178836, 20200000180781, 20200000213999, 20200000215011, 20200000217278, 20200000230132, 20200000252643, 20200000352675, 20200000365285, 20200000386438, 20200000398418, 20200000405670, 20200000418741, 20200000448687, 20200000456397, 20200000460815, 20200000462725, 20200000463359, 20200001706678, 296474127, 370614765, 425073498, 428640326, 473919104, 501220013, 512266298, 584428716, 588013838, 633954666, 643052753, 650460795, 774521832, 791341471, 852801667, 965586129, 1064563184, 1315952208, 20200000034934, 20200000053702, 20200000059345, 20200000069076, 20200000092640, 20200000095162, 20200000098180, 20200000114420, 20200000165375, 20200000177498, 20200000222342, 20200000237746, 20200000242926, 20200000253092, 20200000256092, 20200000260279, 20200000359006, 20200000367714, 20200000368109, 20200000377456, 20200000385644, 20200000385779, 20200000397595, 20200000408093, 20200000414009, 20200000432409, 20200000434117, 20200000435890, 20200000443877, 20200000448213, 20200000453332, 20200000453708, 20200000460446, 422140379, 422349667, 429851711, 442705759, 480645514, 484090401, 550902799, 566294584, 603091649, 631028602, 637897973, 638218744, 689180123, 855627886, 871230822, 908639810, 909480609, 913292932, 970399837, 980274086, 999215145, 1061116191, 1065271333, 1177956445, 1181024195, 1183177580, 1248456559, 20200000026547, 20200000042371, 20200000052226, 20200000087199, 20200000097849, 20200000110249, 20200000110790, 20200000118324, 20200000145036, 20200000146290, 20200000164292, 20200000166500, 20200000167056, 20200000184191, 20200000189955, 20200000209143, 20200000233727, 20200000249581, 20200000250622, 20200000266917, 20200000273391, 20200000360464, 20200000380531, 20200000398209, 20200000399682, 20200000401498, 20200000426365, 20200000432031, 20200000449839, 20200000451066, 20200000451851, 20200000460562, 20200000463792, 20200002013809, 20200002174736, 348144377, 370070561, 384447747, 434756152, 706857885, 852912420, 903183877, 907759227, 920936672, 938082827, 991004934, 1038796140, 1072276976, 1117365978, 1154065290, 1258026834, 20200000004927, 20200000015083, 20200000015167, 20200000019704, 20200000023823, 20200000037985, 20200000041786, 20200000042698, 20200000046242, 20200000053306, 20200000061223, 20200000081511, 20200000100543, 20200000132750, 20200000137364, 20200000148739, 20200000159986, 20200000165141, 20200000169361, 20200000180448, 20200000187176, 20200000222659, 20200000241112, 20200000244971, 20200000257276, 20200000280005, 20200000356865, 20200000371518, 20200000376834, 20200000394250, 20200000405282, 20200000409293, 20200000418832, 20200000424309, 20200000436106, 20200000441248, 20200000457113, 20200000457254, 20200000465730, 20200001448734, 329963928, 364895318, 376888389, 431065024, 473061653, 493849232, 507112674, 643827754, 778472002, 1021191431, 1029107432, 1032381310, 1060102144, 1163346856, 20200000001008, 20200000017488, 20200000047024, 20200000055036, 20200000056907, 20200000060113, 20200000066717, 20200000081376, 20200000111124, 20200000131537, 20200000213835, 20200000243438, 20200000244553, 20200000259773, 20200000274095, 20200000280764, 20200000354704, 20200000386681, 20200000392896, 20200000396419, 20200000403534, 20200000404403, 20200000404779, 20200000416475, 20200000416857, 20200000427384, 20200000434592, 20200000455624, 20200003183492, 284288607, 426007025, 449404451, 496869796, 510551395, 548248249, 568762167, 579304566, 624366832, 708142293, 749132901, 767104707, 888695363, 925814105, 934416963, 942752618, 944808246, 1051636959, 1063302971, 1077307224, 1100196084, 1148045577, 1447864760, 20200000061004, 20200000075782, 20200000091404, 20200000118225, 20200000128791, 20200000168026, 20200000224638, 20200000246501, 20200000261281, 20200000263903, 20200000265726, 20200000267141, 20200000362207, 20200000375741, 20200000376657, 20200000379067, 20200000385384, 20200000421519, 20200000431942, 20200000432353, 20200000436589, 20200000440547, 20200000441291, 20200000447181, 20200000453591, 20200000679562, 20200001586933, 20200002294699, 412230864, 438207253, 469487373, 504750621, 510301841, 520356600, 533969469, 613248785, 657055334, 660127957, 875269293, 951744068, 1042901784, 1052123302, 1064459184, 1137027933, 1140935852, 1159334248, 1179267591, 1438149325, 1495587073, 20200000034463, 20200000076584, 20200000082498, 20200000128913, 20200000137932, 20200000176949, 20200000209572, 20200000230375, 20200000234664, 20200000251742, 20200000260159, 20200000381339, 20200000409888, 20200000428681, 20200000428937, 20200000461721, 20200001685867, 20200003183488, 403997434, 415850124, 463417930, 493619333, 525818839, 564874986, 565616828, 570180361, 582654048, 649181106, 719219382, 736535961, 756186884, 768607049, 829736340, 886123071, 918780702, 927014373, 1054869982, 1105855019, 1144810754, 1148719061, 1204063038, 20200000031035, 20200000036108, 20200000058556, 20200000075859, 20200000076151, 20200000076594, 20200000083832, 20200000123933, 20200000129160, 20200000131129, 20200000160317, 20200000201379, 20200000220173, 20200000231302, 20200000242499, 20200000248686, 20200000257390, 20200000356149, 20200000358744, 20200000366136, 20200000379674, 20200000406419, 20200000414673, 20200000436541, 20200000437588, 20200000438605, 20200000441361, 20200000450886, 20200000459645, 20200000459684, 20200000461517, 20200002027447, 364147881, 460939125, 537263603, 540382946, 556826928, 561048152, 564540143, 586634331, 735128536, 739078122, 773178141, 824421012, 859831864, 907611774, 923143984, 930574573, 992347810, 1002597082, 1004316133, 1020119327, 1023365581, 1034018838, 1064962398, 1065980436, 1395141762, 20200000011648, 20200000012731, 20200000019158, 20200000021151, 20200000040027, 20200000045882, 20200000077910, 20200000101659, 20200000142235, 20200000149041, 20200000159969, 20200000173703, 20200000176008, 20200000206913, 20200000233212, 20200000233766, 20200000247622, 20200000255871, 20200000258274, 20200000262618, 20200000271519, 20200000372966, 20200000382973, 20200000385442, 20200000407317, 20200000421375, 20200000426764, 20200000429102, 20200000432015, 20200000439817, 20200000441951, 20200000449860, 20200002314548, 346996240, 371338715, 395332488, 434090820, 462678375, 489803614, 490650819, 532564474, 547953205, 640364616, 771858109, 789356890, 806401039, 895913573, 1071811770, 1448125766, 1469001628, 20200000004410, 20200000021806, 20200000052515, 20200000087084, 20200000090947, 20200000103179, 20200000104270, 20200000119677, 20200000122874, 20200000142077, 20200000163750, 20200000165901, 20200000181672, 20200000192425, 20200000205862, 20200000211055, 20200000222284, 20200000230661, 20200000231842, 20200000247309, 20200000252363, 20200000255932, 20200000279296, 20200000281292, 20200000361489, 20200000372094, 20200000382685, 20200000400287, 20200000415610, 20200000439255, 20200000445925, 20200000448221, 20200000451896, 20200000458555, 20200000463816, 20200000634786, 20200001707532, 20200002543139, 288120394, 379968583, 388424049, 411430426, 469337564, 578979413, 581365763, 647627184, 687756852, 729925582, 824281761, 853112979, 906120217, 920327210, 1047264616, 1069803052, 1097172005, 1114471030, 1128761295, 1182345576, 1185523281, 1186270245, 1207645596, 20200000004100, 20200000074307, 20200000078693, 20200000082660, 20200000125423, 20200000167823, 20200000169537, 20200000190671, 20200000200194, 20200000206260, 20200000220449, 20200000225244, 20200000246528, 20200000251292, 20200000253726, 20200000261514, 20200000265149, 20200000271526, 20200000358166, 20200000394113, 20200000403473, 20200000408018, 20200000426835, 20200000437948, 20200000443466, 20200000445750, 20200000450060, 20200000464041, 20200002123379, 299216019, 302584613, 344186162, 370299188, 373909837, 378483858, 538182763, 590423668, 592414628, 635707909, 730930974, 737565891, 808122523, 885661130, 919395844, 948958859, 975139176, 976150380, 1060433597, 1104399787, 1114751883, 1116582902, 1164865816, 1183723096, 1192209090, 1226926871, 1236020288, 1269192629, 20200000006792, 20200000020342, 20200000072676, 20200000074575, 20200000077799, 20200000088537, 20200000099989, 20200000106539, 20200000123284, 20200000127355, 20200000193555, 20200000197214, 20200000219150, 20200000259357, 20200000263804, 20200000273260, 20200000350942, 20200000360325, 20200000365620, 20200000384822, 20200000385735, 20200000402601, 20200000409243, 20200000416522, 20200000423961, 20200000425802, 20200000428971, 20200000432478, 20200000435437, 20200000456331, 20200001769263, 20200002339847, 464783893, 473689550, 489673522, 504575083, 545519333, 595563753, 634235735, 694133630, 803892455, 803942354, 879178630, 923072172, 955967118, 971197898, 1049234587, 1056683603, 1065512367, 1067555319, 1072047275, 1142715814, 1177954878, 1185464456, 20200000017376, 20200000028958, 20200000052188, 20200000094003, 20200000094152, 20200000097653, 20200000127959, 20200000140053, 20200000168834, 20200000197846, 20200000231303, 20200000255847, 20200000258064, 20200000282354, 20200000384387, 20200000385672, 20200000386347, 20200000398754, 20200000405308, 20200000418124, 20200000437770, 20200003021717, 300044873, 346776176, 406689200, 412786607, 416769097, 454690785, 460832645, 576443892, 598934033, 768769263, 874809991, 1125276479, 1146860537, 1483428231, 20200000006709, 20200000016867, 20200000017487, 20200000021401, 20200000027743, 20200000058949, 20200000060569, 20200000063260, 20200000072528, 20200000081570, 20200000102146, 20200000107652, 20200000114411, 20200000132019, 20200000141601, 20200000175130, 20200000186911, 20200000211679, 20200000226961, 20200000234135, 20200000253400, 20200000254750, 20200000255319, 20200000277694, 20200000350481, 20200000355928, 20200000360533, 20200000377625, 20200000391641, 20200000410102, 20200000412565, 20200000418153, 20200000422541, 20200000428946, 20200000431526, 20200000431953, 20200000436374, 20200000442109, 20200000451862, 20200000455594, 20200000491774, 20200000759043, 20200002155777, 388688564, 401301276, 464308294, 563546294, 608066898, 665743645, 695805471, 695948278, 840190360, 881155044, 894991908, 894997884, 941620316, 972169659, 1033398935, 1062255050, 1479504067, 20200000004393, 20200000064082, 20200000085884, 20200000096960, 20200000098442, 20200000100997, 20200000101278, 20200000146099, 20200000158213, 20200000165733, 20200000168747, 20200000174066, 20200000187885, 20200000225023, 20200000230411, 20200000231474, 20200000236959, 20200000244448, 20200000249276, 20200000265642, 20200000265773, 20200000410509, 20200000414646, 20200000428938, 20200000441238, 20200000445066, 20200000445201, 20200001138893, 20200001780688, 364159440, 439873467, 486363584, 486717857, 496004846, 536381662, 545625741, 558433129, 574295698, 587250831, 595298727, 624561823, 662938847, 674998456, 698184821, 740362719, 825650886, 828954046, 880626868, 885823239, 896733158, 904223061, 961187459, 997811381, 1041596399, 1050162071, 1062930994, 1073846056, 1499207728, 20200000002658, 20200000011786, 20200000032862, 20200000102329, 20200000107785, 20200000115064, 20200000120085, 20200000130265, 20200000139089, 20200000140458, 20200000140996, 20200000147326, 20200000165537, 20200000255392, 20200000258544, 20200000276743, 20200000277191, 20200000279926, 20200000352770, 20200000354897, 20200000376799, 20200000388575, 20200000396406, 20200000402749, 20200000416500, 20200000432365, 20200000432710, 20200000439926, 20200000441260, 20200000450954, 20200000457624, 20200000457793, 20200001978171, 20200003043719, 301656026, 329127297, 368305725, 370503676, 392590060, 471341991, 488305689, 564315961, 720143823, 720288164, 893393414, 893927401, 931936670, 949754553, 1028610115, 1033801524, 1074889855, 1124344384, 1185364768, 1506716215, 20200000012968, 20200000017071, 20200000018760, 20200000038036, 20200000065104, 20200000072167, 20200000075256, 20200000075287, 20200000124069, 20200000132698, 20200000137649, 20200000161874, 20200000168536, 20200000192894, 20200000201373, 20200000230934, 20200000253211, 20200000282815, 20200000350758, 20200000360113, 20200000360162, 20200000372449, 20200000423460, 20200000427003, 20200000432722, 20200000448698, 20200000456360, 20200000456785, 20200000457489, 284741179, 361309726, 362527234, 404765773, 420285237, 435890283, 436613066, 438596432, 463869460, 543186831, 652077009, 657215015, 672150402, 711407854, 723795263, 799406905, 890378044, 908511009, 914360595, 947433990, 962405601, 966021394, 1070255846, 1071814364, 1088756849, 1093131935, 1099105909, 1438978488, 20200000025682, 20200000029236, 20200000037764, 20200000053478, 20200000063456, 20200000103024, 20200000104386, 20200000119366, 20200000142185, 20200000167689, 20200000185789, 20200000203845, 20200000212940, 20200000219625, 20200000229859, 20200000231954, 20200000264109, 20200000280286, 20200000280767, 20200000352217, 20200000395694, 20200000403732, 20200000404807, 20200000408932, 20200000418335, 20200000425901, 20200000434096, 20200000440113, 20200000440846, 20200002515671, 426283261, 494833223, 609704981, 767319014, 838455884, 927559995, 975023016, 980651497, 1008540661, 1042320103, 1045516045, 1054967219, 1076700027, 1082965369, 1090365192, 1108780504, 1116609570, 1158919754, 1176970599, 1485512615, 1502104710, 20200000053458, 20200000068923, 20200000070179, 20200000091503, 20200000105097, 20200000117126, 20200000143331, 20200000180266, 20200000190660, 20200000196260, 20200000203396, 20200000215037, 20200000218344, 20200000240242, 20200000244461, 20200000248696, 20200000255843, 20200000264028, 20200000376204, 20200000376691, 20200000378008, 20200000407234, 20200000418567, 20200000420233, 20200000421001, 20200000430141, 20200000434408, 20200000436441, 20200000437582, 20200000443205, 20200000446802, 20200000449109, 20200000459655, 20200000459836, 20200002080896, 20200002256455, 20200002625146, 403232367, 470867961, 474711840, 512979868, 537040244, 617390170, 620112416, 734464274, 820934073, 834393815, 857766057, 923086390, 958763157, 979599202, 1049204901, 1087441725, 20200000016920, 20200000017473, 20200000017781, 20200000018631, 20200000051275, 20200000065167, 20200000065977, 20200000114309, 20200000134406, 20200000148797, 20200000163143, 20200000174556, 20200000195311, 20200000207073, 20200000239511, 20200000251111, 20200000265580, 20200000368384, 20200000379144, 20200000401802, 20200000406872, 20200000434105, 20200000446814, 20200000447495, 20200000452556, 20200000452618, 20200000461324, 366322908, 376412160, 400656481, 413515229, 429314901, 465278719, 483049169, 527445936, 537197386, 550318260, 598967192, 1045165396, 1185773124, 1253414840, 1499200100, 20200000004544, 20200000027798, 20200000028629, 20200000054030, 20200000073369, 20200000105434, 20200000105446, 20200000107731, 20200000134636, 20200000170332, 20200000175191, 20200000175836, 20200000197956, 20200000204357, 20200000229320, 20200000237346, 20200000238362, 20200000238897, 20200000281518, 20200000282082, 20200000365152, 20200000382915, 20200000386401, 20200000394193, 20200000407183, 20200000407213, 20200000408529, 20200000414650, 20200000431779, 20200000432191, 20200000434353, 20200000438012, 20200000439483, 20200000449861, 20200000454063, 20200000591562, 281935788, 352134875, 409838725, 428912452, 530246030, 533412119, 551538675, 601831815, 646489967, 655596915, 669575206, 689279040, 897574452, 932816415, 973482525, 988318940, 1012294579, 1017717218, 1128135720, 1157869791, 1184829047, 1502521386, 20200000001033, 20200000019621, 20200000035212, 20200000041183, 20200000046789, 20200000073961, 20200000093126, 20200000106653, 20200000160173, 20200000174565, 20200000211358, 20200000216191, 20200000221554, 20200000227104, 20200000229600, 20200000231832, 20200000391599, 20200000400035, 20200000417688, 20200000420689, 20200000422381, 20200000422677, 20200000441575, 20200000441744, 20200000462867, 20200003158391, 339158729, 444440079, 477962962, 486198800, 504292416, 529096189, 593715088, 605841374, 610049013, 610947968, 645859810, 708869281, 721677994, 838500730, 870867425, 911803869, 938731156, 994674676, 1046846443, 1058526204, 1112690545, 1120294802, 1462162815, 20200000019647, 20200000043710, 20200000060112, 20200000085782, 20200000098293, 20200000115309, 20200000117025, 20200000129861, 20200000140674, 20200000147272, 20200000149627, 20200000150958, 20200000161981, 20200000178822, 20200000206344, 20200000209562, 20200000238027, 20200000246819, 20200000261478, 20200000267708, 20200000272401, 20200000390275, 20200000395086, 20200000428821, 20200000432857, 20200000437078, 20200000444721, 20200000451063, 20200000451176, 20200000456942, 20200000459887, 20200000464170, 414835676, 452055711, 482669234, 489265199, 493868874, 496199099, 511840217, 524622747, 594819551, 604752655, 694576020, 707430013, 716854054, 741889923, 890902210, 908047866, 936669161, 979967712, 1001286466, 1015924630, 1023146677, 1039256544, 1062878207, 1166454845, 1168165306, 20200000040384, 20200000076817, 20200000110175, 20200000125777, 20200000128422, 20200000159292, 20200000190976, 20200000200456, 20200000209607, 20200000223098, 20200000223643, 20200000227960, 20200000240349, 20200000252878, 20200000279951, 20200000283695, 20200000362855, 20200000374757, 20200000406236, 20200000414954, 20200000418627, 20200000421462, 20200000423492, 20200000432360, 20200000433967, 20200000441169, 20200000452133, 20200000460555, 282935706, 372515745, 389279827, 403858572, 410229362, 430616332, 447815630, 475820434, 567198973, 576649830, 634968670, 638689075, 653295053, 667810666, 739095641, 779096042, 827863198, 888640563, 894552505, 955697227, 1021469703, 1100331256, 1151680145, 1158676006, 1163391888, 1188193346, 20200000002799, 20200000013910, 20200000032226, 20200000040305, 20200000046324, 20200000063603, 20200000076870, 20200000078148, 20200000109095, 20200000112532, 20200000121720, 20200000128342, 20200000146009, 20200000157371, 20200000164390, 20200000218784, 20200000218827, 20200000219864, 20200000245479, 20200000252191, 20200000257434, 20200000263102, 20200000267604, 20200000270230, 20200000278339, 20200000281214, 20200000354967, 20200000368623, 20200000398403, 20200000399692, 20200000429422, 20200000438898, 20200000462243, 20200002225019, 20200002502630, 363660568, 404513525, 416753449, 468312752, 506556769, 506627515, 510916181, 573992498, 738480930, 781212171, 781559722, 900006134, 917661628, 957345067, 974021529, 976534404, 1017626345, 1047226782, 1116790278, 1121157685, 1499199631, 20200000024925, 20200000028591, 20200000034693, 20200000046814, 20200000066200, 20200000115371, 20200000116056, 20200000136210, 20200000158457, 20200000174664, 20200000187656, 20200000188321, 20200000204441, 20200000211429, 20200000214058, 20200000245324, 20200000263399, 20200000359666, 20200000361285, 20200000374910, 20200000375269, 20200000389471, 20200000392850, 20200000399196, 20200000406894, 20200000434450, 20200000442669, 20200000449978, 20200000455253, 20200000456154, 20200000460603, 20200002010758, 20200002555990, 20200002563382, 379983299, 412798909, 450155542, 538212549, 543032054, 549226021, 583976519, 635750965, 790203750, 880100350, 887947640, 893677035, 1036988193, 1042608915, 1094724185, 1173516699, 20200000018507, 20200000026277, 20200000098026, 20200000106397, 20200000107067, 20200000113999, 20200000120483, 20200000126394, 20200000147287, 20200000149532, 20200000153002, 20200000163673, 20200000170896, 20200000192701, 20200000215225, 20200000218442, 20200000220574, 20200000240269, 20200000250903, 20200000253788, 20200000258021, 20200000263234, 20200000274005, 20200000384416, 20200000407756, 20200000422663, 20200000427838, 20200000443916, 20200000450191, 20200000452348, 20200000454057, 20200000457480, 20200001443803, 20200001757648, 20200001797396, 20200002158904, 20200002748877, 20200003077907, 317777108, 327630330, 331975235, 376694253, 432858701, 437760529, 505945259, 602773895, 664550975, 665716444, 737480340, 892521917, 1003822659, 1076799933, 1203445491, 20200000028587, 20200000035256, 20200000046782, 20200000047219, 20200000076910, 20200000101934, 20200000103048, 20200000103250, 20200000107014, 20200000126740, 20200000128545, 20200000137542, 20200000144857, 20200000147976, 20200000207798, 20200000232321, 20200000277740, 20200000277846, 20200000279551, 20200000354998, 20200000360498, 20200000377527, 20200000404339, 20200000409857, 20200000414278, 20200000423589, 20200000432860, 20200000435048, 20200000437129, 20200000440517, 20200000441412, 20200000444810, 20200000444977, 20200000449637, 20200000452303, 20200002045930, 20200002595177, 20200003122343, 366572045, 419425457, 487119327, 495705977, 505124220, 505544063, 536051560, 577993765, 599922935, 641291615, 727466621, 843756298, 853747336, 880629270, 891378056, 895011486, 921022358, 1072395467, 1114430941, 20200000008386, 20200000018343, 20200000026930, 20200000030089, 20200000032884, 20200000049788, 20200000052284, 20200000079055, 20200000082072, 20200000084718, 20200000105519, 20200000115865, 20200000124784, 20200000143035, 20200000152654, 20200000169078, 20200000183212, 20200000191296, 20200000197851, 20200000202129, 20200000208598, 20200000231753, 20200000268918, 20200000351723, 20200000363876, 20200000371950, 20200000388910, 20200000391094, 20200000410962, 20200000424448, 20200000426099, 20200000441682, 20200000456310, 20200000601393, 20200003039626, 377342622, 384199656, 393521916, 396082453, 469369175, 495077699, 515909521, 523642255, 529096064, 532046195, 534558776, 719525810, 749057895, 917661035, 970389495, 1013819832, 1107822876, 1110785293, 20200000031468, 20200000037041, 20200000059908, 20200000060686, 20200000083187, 20200000108990, 20200000127755, 20200000152538, 20200000161460, 20200000178219, 20200000210752, 20200000213941, 20200000270659, 20200000358544, 20200000367776, 20200000375180, 20200000399945, 20200000407161, 20200000408172, 20200000414861, 20200000430052, 20200000431283, 20200000438703, 20200000446791, 20200000449551, 20200000460483, 293778748, 317799861, 330773889, 347798982, 449942749, 504871113, 512350210, 524509185, 548572622, 592297580, 687451368, 945274928, 964326593, 1058370552, 20200000006060, 20200000010860, 20200000056276, 20200000072457, 20200000085838, 20200000093056, 20200000105878, 20200000148897, 20200000152631, 20200000168311, 20200000211629, 20200000219168, 20200000234316, 20200000258592, 20200000263069, 20200000281458, 20200000282893, 20200000350841, 20200000353637, 20200000360939, 20200000362382, 20200000368057, 20200000376044, 20200000401604, 20200000422228, 20200000426127, 20200000426137, 20200000430775, 20200000432028, 20200000434169, 20200000440053, 20200000447247, 20200000447432, 20200000457463, 513188658, 604836034, 628677149, 653453530, 733299948, 887138564, 909351158, 972003056, 998178668, 1002043426, 1045960442, 1047848937, 1072221091, 1130369309, 1189781891, 1450378764, 20200000015485, 20200000039465, 20200000044002, 20200000060176, 20200000093536, 20200000124399, 20200000138123, 20200000153949, 20200000157536, 20200000189684, 20200000197717, 20200000201436, 20200000210290, 20200000220637, 20200000247712, 20200000268479, 20200000283292, 20200000356408, 20200000359141, 20200000373887, 20200000374909, 20200000389973, 20200000392355, 20200000402636, 20200000409525, 20200000414674, 20200000418292, 20200000424563, 20200000427625, 20200000429425, 20200000435760, 20200000440214, 20200000443872, 20200000446700, 20200000457504, 20200000458203, 20200000462102, 282614216, 315577859, 319881193, 328608539, 438164450, 440045374, 441457218, 495843728, 595831580, 624687337, 651887277, 763452849, 784458141, 839263141, 920402079, 1009336960, 1011822540, 1038376578, 1042301779, 1077617036, 1098099856, 1102953355, 1178542436, 1456733988, 1498880165, 20200000007565, 20200000022932, 20200000026813, 20200000057479, 20200000061925, 20200000065502, 20200000073426, 20200000105068, 20200000106184, 20200000125562, 20200000126185, 20200000134794, 20200000146782, 20200000173736, 20200000196274, 20200000196312, 20200000211294, 20200000221031, 20200000254058, 20200000350843, 20200000365623, 20200000367184, 20200000375469, 20200000380928, 20200000392707, 20200000406488, 20200000407001, 20200000422385, 20200000423528, 20200000423916, 20200000431828, 20200000439298, 20200000442230, 20200000457496, 20200000461282, 283436103, 431946152, 455190486, 456567951, 489448322, 501940267, 589798479, 661267609, 684332334, 796274158, 859204347, 878577184, 919218502, 936966570, 953876014, 959908019, 993005068, 1034630958, 1080225654, 1130835003, 1135866296, 1215779598, 20200000029920, 20200000046629, 20200000049099, 20200000064706, 20200000075062, 20200000090245, 20200000132327, 20200000133652, 20200000178939, 20200000194617, 20200000197573, 20200000198499, 20200000199978, 20200000208062, 20200000249757, 20200000280155, 20200000356977, 20200000358890, 20200000361024, 20200000404243, 20200000413979, 20200000414868, 20200000426611, 20200000429501, 20200003105018, 451658773, 496499395, 521963573, 581983831, 582486077, 611079116, 623193844, 659425518, 847985808, 886621471, 917212879, 937789710, 946011498, 983143183, 1043589663, 1106643066, 1398185750, 1499265907, 20200000013079, 20200000013169, 20200000015987, 20200000024386, 20200000040238, 20200000061049, 20200000061876, 20200000079195, 20200000098031, 20200000111709, 20200000157934, 20200000180441, 20200000213283, 20200000214245, 20200000237715, 20200000238233, 20200000253975, 20200000280708, 20200000359662, 20200000362675, 20200000375779, 20200000394772, 20200000397311, 20200000404777, 20200000413828, 20200000421756, 20200000428095, 20200000430759, 20200002314129, 333903271, 496492874, 498493962, 507429840, 513227836, 519200450, 539124565, 632927533, 645746786, 646336340, 657821775, 783206968, 797809037, 933135994, 945539291, 964163407, 977126486, 1058295660, 1072527403, 1085468464, 1176642064, 20200000011719, 20200000014080, 20200000036515, 20200000041094, 20200000079745, 20200000097921, 20200000100449, 20200000105804, 20200000113468, 20200000118345, 20200000120403, 20200000143265, 20200000143919, 20200000163250, 20200000174021, 20200000177285, 20200000186694, 20200000190284, 20200000192569, 20200000194262, 20200000210814, 20200000219360, 20200000225983, 20200000240821, 20200000256011, 20200000270870, 20200000277341, 20200000353935, 20200000361979, 20200000377312, 20200000403509, 20200000411648, 20200000413669, 20200000416325, 20200000423576, 20200000425449, 20200000434337, 20200000441300, 20200000445226, 20200000454055, 20200000455641, 363724991, 507895385, 568491494, 572395608, 577320385, 585661281, 610340475, 702895523, 789969216, 890637494, 930829303, 956622006, 1115911787, 1116380188, 20200000013982, 20200000016853, 20200000038695, 20200000049860, 20200000052102, 20200000070210, 20200000078355, 20200000095371, 20200000106325, 20200000113866, 20200000119789, 20200000144814, 20200000156956, 20200000167002, 20200000172315, 20200000175192, 20200000186950, 20200000231872, 20200000234549, 20200000236919, 20200000246413, 20200000254600, 20200000270595, 20200000274231, 20200000280694, 20200000282027, 20200000354390, 20200000387316, 20200000416405, 20200000416581, 20200000422193, 20200000422734, 20200000428352, 20200000441243, 20200000445179, 20200000447392, 20200000452964, 20200000458314, 20200001418273, 333710667, 364467713, 373743686, 430537586, 443369807, 454629399, 512680238, 562184149, 566199224, 660982355, 699494089, 706398884, 788990886, 798559502, 885271158, 903671824, 916130071, 916869395, 917667294, 942494098, 1010762255, 1010882968, 1048855522, 1061907653, 1119482395, 1137683736, 1402540811, 20200000008526, 20200000011780, 20200000038955, 20200000039747, 20200000042882, 20200000061124, 20200000080086, 20200000093112, 20200000103860, 20200000107750, 20200000114475, 20200000127154, 20200000146973, 20200000174270, 20200000196101, 20200000201272, 20200000218218, 20200000223803, 20200000224633, 20200000227754, 20200000241449, 20200000244353, 20200000251848, 20200000275232, 20200000281481, 20200000358154, 20200000362313, 20200000370014, 20200000372689, 20200000372896, 20200000377357, 20200000382347, 20200000387259, 20200000391590, 20200000403081, 20200000463925, 20200001699262, 317389691, 412163953, 435357526, 447689011, 448633376, 454781476, 458588475, 509457434, 544398031, 547305617, 566685699, 580175049, 605681399, 633163770, 668677601, 749113886, 784439432, 845918296, 852197796, 886076992, 923760656, 939272201, 993674943, 1007247477, 1008692660, 1011501094, 1017261655, 1029143181, 1042683863, 1137328689, 1187804884, 20200000014813, 20200000019884, 20200000026682, 20200000046093, 20200000054570, 20200000056625, 20200000066038, 20200000095049, 20200000121941, 20200000129693, 20200000131133, 20200000165312, 20200000205493, 20200000225931, 20200000233417, 20200000251182, 20200000261588, 20200000264395, 20200000355027, 20200000357917, 20200000371777, 20200000381438, 20200000398407, 20200000407873, 20200000409828, 20200000414693, 20200000420104, 20200000420709, 20200000421091, 20200000435747, 20200000445986, 20200000453934, 20200000457513, 20200001485164, 327783342, 329913454, 369094718, 388483817, 394874573, 424183595, 439910003, 512142109, 555676407, 556164350, 573512990, 594843575, 634113809, 676487207, 684042521, 718153412, 835969177, 913399756, 972319818, 998320605, 1049235852, 1054355169, 1136993233, 1157858872, 1168271125, 1176053552, 20200000012214, 20200000041964, 20200000065765, 20200000066464, 20200000068106, 20200000083820, 20200000084453, 20200000085158, 20200000085593, 20200000098757, 20200000104112, 20200000120843, 20200000153032, 20200000172785, 20200000175832, 20200000176117, 20200000247669, 20200000248445, 20200000280881, 20200000371872, 20200000373735, 20200000382258, 20200000404280, 20200000411964, 20200000414895, 20200000437464, 20200000443072, 20200000443974, 20200002571817, 20200003045765, 317703490, 391432693, 413971331, 430399296, 430408238, 493976598, 547702041, 552228536, 587107345, 608188610, 638497167, 650028066, 666465255, 699906836, 729430378, 886281967, 994859986, 998176001, 1156447726, 1273670388, 20200000010866, 20200000048072, 20200000066164, 20200000082225, 20200000108284, 20200000126397, 20200000138983, 20200000145859, 20200000161270, 20200000161645, 20200000165875, 20200000178848, 20200000203811, 20200000211764, 20200000215337, 20200000219112, 20200000269221, 20200000357813, 20200000361369, 20200000364702, 20200000370719, 20200000377432, 20200000380475, 20200000381503, 20200000386436, 20200000387724, 20200000404476, 20200000417837, 20200000438140, 20200000442391, 20200000449367, 20200000457596, 20200000461836, 20200002864302, 302049354, 342790226, 554149554, 891362701, 931254212, 965137032, 991153141, 1034521018, 1077626197, 1141876504, 1160056295, 1171417784, 20200000027176, 20200000031813, 20200000040404, 20200000048523, 20200000053645, 20200000063748, 20200000075191, 20200000111640, 20200000115863, 20200000118112, 20200000170466, 20200000181765, 20200000192002, 20200000194070, 20200000215050, 20200000274188, 20200000358225, 20200000374798, 20200000376128, 20200000382566, 20200000405382, 20200000408297, 20200000421163, 20200000422302, 20200000430684, 20200000433174, 20200000440065, 20200000442859, 20200000451698, 20200000459701, 20200000462847, 20200002095254, 374982811, 399211291, 494792609, 512990046, 555936735, 579995876, 767473889, 800650896, 897863326, 944393180, 986041232, 1057844059, 1090829780, 1121463708, 1219873952, 1269652963, 20200000000629, 20200000004765, 20200000039732, 20200000042519, 20200000043052, 20200000043880, 20200000081311, 20200000089193, 20200000102636, 20200000105106, 20200000120630, 20200000142919, 20200000151350, 20200000173907, 20200000210513, 20200000242274, 20200000262319, 20200000276884, 20200000351622, 20200000363703, 20200000385895, 20200000401196, 20200000418253, 20200000429421, 20200000435464, 20200000441308, 20200000445171, 20200000448677, 20200000460604, 326562833, 353596787, 398329664, 416254083, 519308960, 526641427, 567382103, 608206510, 624533261, 654657007, 721639879, 721782955, 835520522, 838736694, 875193091, 926122157, 969061976, 990994624, 1030548464, 1097727390, 1123667089, 1149994032, 20200000015377, 20200000022316, 20200000046950, 20200000062951, 20200000104742, 20200000107948, 20200000112430, 20200000142533, 20200000145128, 20200000146387, 20200000148768, 20200000177741, 20200000184127, 20200000190433, 20200000194328, 20200000198171, 20200000236697, 20200000237239, 20200000237917, 20200000238811, 20200000250637, 20200000352282, 20200000402435, 20200000403650, 20200000404232, 20200000408955, 20200000415759, 20200000418000, 20200000421253, 20200000450519, 421254504, 530493732, 564582898, 570964996, 648548913, 660701147, 899246394, 902262477, 946002855, 955404874, 1089982285, 1099857083, 1156514888, 20200000011027, 20200000014660, 20200000033854, 20200000132579, 20200000140746, 20200000144143, 20200000152566, 20200000169485, 20200000182605, 20200000213650, 20200000213860, 20200000245996, 20200000264924, 20200000280400, 20200000367432, 20200000380553, 20200000380757, 20200000388945, 20200000396659, 20200000404071, 20200000432901, 20200000444026, 20200000456314, 20200000457850, 20200000459304, 20200000462637, 20200001740061, 20200001803383, 20200002235345, 20200002897693, 336683524, 367939284, 467810884, 587323410, 623367437, 627950311, 670935388, 838886497, 849095637, 883738213, 886697025, 943248631, 994673739, 1009134067, 1028734023, 1091699305, 1157626939, 1175333653, 1450502249, 1499476594, 20200000000107, 20200000004498, 20200000027289, 20200000039968, 20200000051591, 20200000058698, 20200000075141, 20200000086534, 20200000105445, 20200000124507, 20200000128367, 20200000135194, 20200000171184, 20200000183310, 20200000183690, 20200000201917, 20200000226452, 20200000238740, 20200000245347, 20200000248311, 20200000268697, 20200000271276, 20200000272226, 20200000272248, 20200000273046, 20200000357482, 20200000367484, 20200000373355, 20200000386601, 20200000389887, 20200000404380, 20200000420333, 20200000420866, 20200000423869, 20200000425930, 20200000432671, 20200000447169, 20200000457661, 20200002207613, 20200002329339, 301340979, 319354212, 388344396, 389077966, 395858350, 416653133, 449756408, 468391118, 469339346, 476336895, 533809537, 544007664, 567289775, 655619282, 659270490, 664991834, 731041554, 789631430, 797831274, 890716443, 958655957, 1060259009, 1114681193, 1123443736, 1175513706, 1456545486, 20200000013549, 20200000082068, 20200000083436, 20200000159312, 20200000180509, 20200000192586, 20200000197830, 20200000226898, 20200000230732, 20200000231124, 20200000235162, 20200000356452, 20200000387647, 20200000393965, 20200000405022, 20200000409421, 20200000414772, 20200000441974, 20200000443735, 20200000457508, 20200000462977, 363500805, 392550341, 419627319, 489833171, 532499096, 536531953, 547563774, 720126929, 800625885, 821535810, 924695435, 930350602, 944158857, 948165343, 951627022, 1072380719, 1073816197, 1076644474, 1076687244, 1118751636, 20200000001867, 20200000025219, 20200000046110, 20200000058715, 20200000061224, 20200000084928, 20200000095745, 20200000097125, 20200000119329, 20200000124383, 20200000124585, 20200000151199, 20200000156066, 20200000167721, 20200000232925, 20200000256016, 20200000265453, 20200000271639, 20200000280325, 20200000280457, 20200000280565, 20200000359252, 20200000359323, 20200000391354, 20200000391742, 20200000404386, 20200000428197, 20200000428922, 20200000442965, 20200000444947, 20200000445622, 20200000453361, 20200000472933, 20200003190736, 370339723, 370472970, 388488613, 438203091, 530829008, 540128038, 574412710, 633488924, 643496868, 652041723, 666313522, 705904963, 747541884, 826523703, 878855875, 879387064, 883149380, 898072367, 924373886, 946848480, 953504696, 1082280681, 1119305026, 1161423270, 1181829729, 20200000019686, 20200000039775, 20200000059743, 20200000060480, 20200000120882, 20200000194920, 20200000199543, 20200000212665, 20200000229975, 20200000232924, 20200000267418, 20200000277057, 20200000360762, 20200000361604, 20200000372417, 20200000386608, 20200000411142, 20200000414125, 20200000420205, 20200000422057, 20200000424536, 20200000434795, 20200000435163, 20200000441316, 20200000442856, 20200000449519, 20200000450995, 20200000653424, 20200001496752, 20200001727171, 20200002306065, 20200003048421, 381064973, 412934791, 417777231, 445553058, 458850997, 668416040, 722428252, 778829717, 838177019, 871199775, 907405413, 957300245, 981145589, 1002340615, 1041951266, 1071913738, 1159273771, 1176492470, 1501487442, 1504927898, 20200000009767, 20200000031134, 20200000034487, 20200000056311, 20200000059595, 20200000089222, 20200000108865, 20200000125166, 20200000133480, 20200000135137, 20200000158736, 20200000170322, 20200000170325, 20200000182533, 20200000220335, 20200000222794, 20200000232976, 20200000237779, 20200000248790, 20200000269540, 20200000273014, 20200000281042, 20200000358172, 20200000374744, 20200000377016, 20200000382453, 20200000389928, 20200000394615, 20200000406435, 20200000417334, 20200000424627, 20200000439596, 20200000441644, 20200000450265, 20200002830076, 393313223, 487624620, 492594590, 529829305, 534568162, 552039496, 572537866, 607127533, 623158699, 640702076, 680885433, 859078808, 879920570, 893521440, 896319095, 984444061, 1017417900, 1036661603, 1050538513, 1073837983, 1073936467, 1161656633, 1212466150, 20200000013344, 20200000015822, 20200000019765, 20200000026297, 20200000041181, 20200000042378, 20200000072212, 20200000078828, 20200000089680, 20200000099445, 20200000101014, 20200000104883, 20200000147389, 20200000165204, 20200000171289, 20200000179732, 20200000188805, 20200000206058, 20200000209314, 20200000231774, 20200000236870, 20200000247989, 20200000260050, 20200000262191, 20200000273067, 20200000273943, 20200000377468, 20200000380909, 20200000392715, 20200000393431, 20200000395273, 20200000404731, 20200000422418, 20200000446694, 20200000451082, 20200000462775, 20200001606097, 414706506, 436737730, 489221652, 513894363, 741224931, 886619142, 926202053, 981796690, 996613318, 1089336971, 1114921009, 1200305103, 20200000010632, 20200000020030, 20200000023976, 20200000026904, 20200000033039, 20200000067667, 20200000084181, 20200000085996, 20200000089648, 20200000091203, 20200000093773, 20200000104037, 20200000121764, 20200000129331, 20200000131155, 20200000133985, 20200000134056, 20200000157652, 20200000159416, 20200000190231, 20200000193599, 20200000215717, 20200000235653, 20200000258328, 20200000263691, 20200000354034, 20200000364159, 20200000386388, 20200000399051, 20200000407226, 20200000425037, 20200000427690, 20200000440462, 20200000445924, 20200000446745, 20200000455529, 20200000456157, 20200002880141, 556867879, 594753075, 635042671, 694164275, 718985796, 835599320, 840107884, 879835044, 881488675, 906080826, 1010663747, 1090242126, 1093931023, 1103201311, 1158794296, 1174041307, 1211067079, 20200000000514, 20200000021710, 20200000032107, 20200000033584, 20200000045833, 20200000062374, 20200000069026, 20200000139065, 20200000143882, 20200000176627, 20200000191781, 20200000206819, 20200000223874, 20200000236818, 20200000238662, 20200000239320, 20200000239345, 20200000241340, 20200000245425, 20200000355418, 20200000360544, 20200000388777, 20200000401385, 20200000405814, 20200000426108, 20200000428317, 20200000455210, 293622097, 378172082, 447997168, 479587718, 553023476, 586114395, 588325387, 600672481, 640198920, 666267803, 682567194, 691861483, 968744559, 982151676, 1015438644, 1019192721, 1057125583, 1132509649, 1455601546, 20200000000287, 20200000019058, 20200000027106, 20200000054870, 20200000061102, 20200000075795, 20200000077985, 20200000085539, 20200000090132, 20200000095193, 20200000103466, 20200000133039, 20200000165534, 20200000196398, 20200000199534, 20200000202483, 20200000204100, 20200000205541, 20200000263431, 20200000279773, 20200000367073, 20200000373583, 20200000379210, 20200000380135, 20200000381961, 20200000392941, 20200000409170, 20200000424992, 20200000433745, 20200000435938, 20200000451917, 20200000452637, 294710480, 310636441, 326876192, 383967580, 483510517, 635169016, 642207368, 647783374, 665811972, 762512956, 793064317, 794507331, 864880531, 883191610, 890479211, 922314576, 964414696, 992098515, 998990823, 1024506807, 1031681760, 1050745469, 20200000001993, 20200000007059, 20200000031204, 20200000045559, 20200000048565, 20200000078032, 20200000080056, 20200000080539, 20200000080557, 20200000093800, 20200000099648, 20200000105993, 20200000114639, 20200000147634, 20200000168367, 20200000184412, 20200000201506, 20200000222995, 20200000245819, 20200000248962, 20200000275632, 20200000368672, 20200000383015, 20200000391317, 20200000412588, 20200000415320, 20200000421053, 20200000425055, 20200000433631, 20200000441665, 20200000458007, 20200002024596, 432803486, 458424230, 505864483, 521640648, 534529876, 540925500, 545919428, 556544533, 580554002, 605734365, 919743868, 949785353, 969316884, 997710110, 1011000146, 1028295028, 1047146871, 1081079094, 1121971067, 1125643802, 1160688144, 1171070967, 20200000030374, 20200000031809, 20200000043308, 20200000053000, 20200000072305, 20200000089625, 20200000108356, 20200000127423, 20200000132478, 20200000140319, 20200000152691, 20200000180846, 20200000187143, 20200000217998, 20200000222261, 20200000239154, 20200000244754, 20200000251417, 20200000255576, 20200000263015, 20200000270040, 20200000277063, 20200000354588, 20200000363201, 20200000370192, 20200000373845, 20200000390261, 20200000395872, 20200000417651, 20200000422356, 20200000427052, 20200000428559, 20200000430208, 20200000431790, 20200000441869, 20200000462647, 20200002411572, 330769722, 366247306, 388154228, 412610956, 444525889, 496202653, 523002940, 593845428, 638637819, 645675002, 667728512, 836536344, 844061344, 846721148, 959801877, 975035622, 990801834, 993482556, 994933845, 998286883, 999167262, 1126382960, 1134956102, 1157416026, 1173332347, 1488627761, 20200000006753, 20200000013105, 20200000017138, 20200000035404, 20200000054734, 20200000065465, 20200000073496, 20200000081139, 20200000083918, 20200000107995, 20200000108717, 20200000121878, 20200000133929, 20200000136747, 20200000144392, 20200000151359, 20200000152519, 20200000168800, 20200000173586, 20200000205839, 20200000211536, 20200000243547, 20200000264157, 20200000353086, 20200000360608, 20200000371648, 20200000382557, 20200000386623, 20200000391940, 20200000394094, 20200000395590, 20200000395849, 20200000410068, 20200000413643, 20200000413916, 20200000417727, 20200000424019, 20200000429437, 20200000438454, 20200000441957, 20200000444685, 20200000448400, 20200000455088, 20200002553723, 20200002712174, 20200002840842, 20200002886242, 384036344, 390039844, 499146288, 572847748, 579528355, 582896596, 593216138, 659353189, 664575829, 671464704, 720068672, 725215120, 788443699, 798769189, 823499224, 844428198, 906560615, 909017474, 918093169, 931407908, 976303710, 1001247878, 1027779769, 1042597125, 1070107731, 1076806233, 1093190533, 1093833768, 1106558008, 1500413812, 20200000018416, 20200000029832, 20200000046033, 20200000093923, 20200000120924, 20200000155570, 20200000156925, 20200000158288, 20200000164247, 20200000175896, 20200000192283, 20200000215795, 20200000230035, 20200000264108, 20200000269498, 20200000274250, 20200000376997, 20200000382939, 20200000386238, 20200000396932, 20200000407763, 20200000409399, 20200000414921, 20200000424565, 20200000428283, 20200000433133, 20200000444999, 20200000448978, 20200000457021, 20200000461895, 20200000463781, 20200002438024, 20200002803978, 320494150, 363317633, 372703899, 409009761, 412991664, 445124476, 447748715, 530060483, 569846869, 580133295, 922856077, 1028362533, 1089138400, 1229600187, 1483009655, 20200000012096, 20200000023600, 20200000044940, 20200000057838, 20200000058385, 20200000060138, 20200000087076, 20200000103963, 20200000106358, 20200000113404, 20200000116207, 20200000153624, 20200000153955, 20200000155955, 20200000169471, 20200000171410, 20200000177743, 20200000177810, 20200000198241, 20200000206505, 20200000229649, 20200000233424, 20200000269570, 20200000270529, 20200000277088, 20200000384212, 20200000386297, 20200000386319, 20200000398951, 20200000424981, 20200000432658, 20200000433269, 20200000436498, 20200001835770, 20200002884921, 368177365, 384898189, 394038232, 430200224, 498097099, 640111933, 657559811, 748057890, 777310222, 833529733, 877831982, 892318081, 932647943, 1041464932, 1073286211, 1080816579, 1125304438, 1441757823, 20200000043596, 20200000062651, 20200000084551, 20200000097740, 20200000133655, 20200000140157, 20200000144891, 20200000155369, 20200000165852, 20200000173067, 20200000211323, 20200000222081, 20200000236383, 20200000263337, 20200000267942, 20200000358979, 20200000388596, 20200000390309, 20200000402349, 20200000406853, 20200000409408, 20200000412563, 20200000421315, 20200000423230, 20200000442330, 20200000455447, 20200000458542, 20200000461820, 20200001778735, 20200002530338, 392188745, 424560818, 434101127, 492262685, 574203838, 580002794, 635807688, 640564761, 682145503, 711756202, 736787261, 761434745, 763436590, 895875773, 908944801, 937976391, 941421328, 949773739, 1039633033, 1084618775, 1097106674, 1110376793, 1129121760, 1143052259, 1162202343, 20200000029305, 20200000041792, 20200000057231, 20200000058942, 20200000101589, 20200000102407, 20200000114246, 20200000115562, 20200000121066, 20200000126505, 20200000127529, 20200000194013, 20200000280514, 20200000283978, 20200000360936, 20200000361028, 20200000371857, 20200000377465, 20200000427134, 20200000428265, 20200000444864, 20200000447159, 20200000461067, 20200000462843, 20200000463045, 293154336, 319305999, 374993959, 375664105, 377377200, 389210026, 402694658, 422125323, 453260249, 470890348, 476553281, 481150251, 512628478, 558819288, 700637744, 701120903, 799442576, 826440022, 871229402, 923268741, 928106889, 969247886, 979699290, 990532721, 1035635092, 1115415865, 1144364163, 20200000003449, 20200000006176, 20200000033583, 20200000044585, 20200000053032, 20200000055637, 20200000057506, 20200000073521, 20200000074016, 20200000100513, 20200000107962, 20200000116338, 20200000127690, 20200000201186, 20200000208157, 20200000213450, 20200000226736, 20200000226932, 20200000247480, 20200000252843, 20200000274181, 20200000279157, 20200000283593, 20200000350844, 20200000358590, 20200000360142, 20200000362066, 20200000403547, 20200000403572, 20200000406673, 20200000420924, 20200000440971, 20200000453622, 20200000460419, 20200001870060, 20200002436215, 20200002465421, 20200002571015, 294865266, 364204209, 510461370, 520762327, 564161145, 570426586, 635573390, 663820495, 692756899, 705831920, 807999823, 816757489, 860422887, 886533398, 916798283, 917637901, 946087246, 957345047, 981952596, 1024507512, 1065523219, 1105903447, 1111192302, 20200000012739, 20200000021242, 20200000049982, 20200000053303, 20200000064245, 20200000160704, 20200000181608, 20200000187580, 20200000196218, 20200000207347, 20200000208527, 20200000210786, 20200000219790, 20200000224990, 20200000251970, 20200000271600, 20200000359617, 20200000415498, 20200000434145, 20200000441176, 20200000445276, 20200000451181, 20200000452939, 20200000461434, 20200003182628, 304021996, 376025304, 391515347, 426912243, 439880314, 490979746, 509993510, 528363174, 557138109, 572821456, 585512923, 749118884, 794546203, 814977594, 872605537, 912874848, 919132359, 941143328, 961875786, 1039401309, 1068095192, 1091684426, 1232520254, 20200000015526, 20200000107935, 20200000117979, 20200000151737, 20200000157144, 20200000166470, 20200000202567, 20200000219624, 20200000222993, 20200000269310, 20200000377509, 20200000383864, 20200000389029, 20200000389371, 20200000390845, 20200000415223, 20200000431733, 20200000439591, 20200000441274, 20200000445742, 20200000448208, 20200000450043, 20200000452377, 20200000463794, 20200002017774, 363998953, 407963919, 429952974, 462182115, 521671873, 545229893, 557545294, 599877646, 625257520, 748608884, 919509353, 1039715745, 1044805956, 1093728958, 1094930513, 1130901077, 1168304162, 1199134811, 1444718487, 20200000005081, 20200000017749, 20200000040091, 20200000040607, 20200000115271, 20200000117217, 20200000121694, 20200000132283, 20200000145139, 20200000158341, 20200000164504, 20200000164649, 20200000169196, 20200000189439, 20200000223486, 20200000229486, 20200000278763, 20200000351423, 20200000362635, 20200000375191, 20200000375808, 20200000393005, 20200000393830, 20200000397205, 20200000397619, 20200000398579, 20200000402419, 20200000446765, 20200001550337, 20200002308005, 20200002515368, 20200003054826, 305172250, 376129315, 387136454, 484081903, 653967729, 680819774, 739212016, 871849480, 902585800, 905852173, 916273469, 918098375, 940070484, 941057494, 959619034, 981480244, 1018985924, 1095572547, 1110291326, 1119681672, 1147297267, 1160873391, 1445810571, 1495286186, 20200000005923, 20200000014061, 20200000017890, 20200000029950, 20200000032358, 20200000034695, 20200000053906, 20200000061711, 20200000072893, 20200000124248, 20200000152341, 20200000152366, 20200000171837, 20200000209593, 20200000212527, 20200000234674, 20200000265058, 20200000273109, 20200000360262, 20200000385738, 20200000386473, 20200000408456, 20200000416596, 20200000416741, 20200000419848, 20200000422158, 20200000423308, 20200000424483, 20200000429208, 20200000443428, 20200000457896, 376412660, 418662204, 468566852, 550971517, 554425888, 570555750, 575658129, 603527166, 661982948, 694995124, 718082838, 778145928, 783573626, 814628773, 816177666, 958925654, 991018252, 1043579926, 1047457770, 1071273233, 1121782467, 1133510322, 1177609010, 1180503816, 20200000007969, 20200000050624, 20200000054910, 20200000057191, 20200000061015, 20200000067460, 20200000080036, 20200000083599, 20200000089083, 20200000107911, 20200000112393, 20200000138652, 20200000167483, 20200000171303, 20200000208392, 20200000211861, 20200000243646, 20200000255584, 20200000256671, 20200000277514, 20200000350926, 20200000357577, 20200000359789, 20200000360993, 20200000397373, 20200000398584, 20200000399654, 20200000406467, 20200000421353, 20200000425084, 20200000430038, 20200000434151, 20200000437468, 20200000439979, 20200000442507, 20200000454829, 284797552, 284980812, 363553057, 451697322, 498464830, 640546476, 719135055, 852530891, 858283863, 870601864, 881270303, 883858344, 886565180, 926043030, 933038669, 973482987, 1028623510, 1175955197, 1176394261, 1226311841, 1327349153, 1497678508, 20200000003617, 20200000007663, 20200000010232, 20200000031005, 20200000037838, 20200000046060, 20200000059147, 20200000072658, 20200000093650, 20200000163141, 20200000169468, 20200000172550, 20200000176530, 20200000199785, 20200000201770, 20200000206073, 20200000206930, 20200000217418, 20200000235146, 20200000246877, 20200000251105, 20200000272095, 20200000275097, 20200000365073, 20200000380199, 20200000383421, 20200000385302, 20200000416261, 20200000434303, 20200000453559, 20200000464305, 20200003064615, 452974777, 453960878, 478830752, 486950401, 586449534, 665038527, 673013274, 771269254, 778692530, 811134847, 859084930, 899593829, 909215075, 940006911, 958157733, 1037414606, 1084631296, 1094591345, 1104945393, 1185863882, 1413015626, 20200000012879, 20200000018153, 20200000030442, 20200000035418, 20200000035854, 20200000037920, 20200000044235, 20200000083104, 20200000106596, 20200000113341, 20200000115447, 20200000127375, 20200000137903, 20200000155254, 20200000155979, 20200000160161, 20200000203555, 20200000225962, 20200000232310, 20200000232314, 20200000254744, 20200000259490, 20200000277096, 20200000390434, 20200000397125, 20200000397746, 20200000411697, 20200000417303, 20200000436569, 20200000441805, 20200000447526, 20200000453375, 20200002824584, 20200003183449, 358308707, 386704683, 436105486, 476603267, 489101769, 615563599, 757843896, 828807648, 833517462, 841098932, 957471210, 969589921, 1005276342, 1072374524, 1115730443, 1118875729, 1144412562, 1156110079, 1300060826, 20200000010756, 20200000058399, 20200000064857, 20200000068978, 20200000082125, 20200000083389, 20200000104682, 20200000105531, 20200000117507, 20200000123259, 20200000157174, 20200000200551, 20200000212179, 20200000214276, 20200000222871, 20200000258451, 20200000367135, 20200000379782, 20200000407611, 20200000410785, 20200000415305, 20200000436309, 20200000442071, 20200000447187, 20200000448589, 20200000449562, 20200000455448, 20200002121885, 20200002493191, 346782677, 382418196, 475905823, 478601195, 524359189, 543821889, 575812869, 576836614, 603577831, 635451623, 685973630, 692081445, 694718787, 708600202, 754659884, 783010196, 791211390, 838849800, 903749076, 969040517, 992051355, 1007115851, 1010999283, 1123631069, 1151062897, 1228038935, 20200000018649, 20200000052176, 20200000070883, 20200000084281, 20200000089515, 20200000133120, 20200000215184, 20200000226428, 20200000246956, 20200000268193, 20200000358516, 20200000369767, 20200000371371, 20200000375627, 20200000391721, 20200000400220, 20200000410786, 20200000417841, 20200000421114, 20200000429709, 20200000434475, 20200000444582, 20200000462721, 20200001822358, 364739177, 390553006, 425746946, 463471155, 515982292, 518969436, 590391110, 615422353, 621370271, 664306853, 681370499, 749078962, 791513983, 805477702, 838579136, 1027688889, 1052343783, 1080405775, 1154932397, 1184828464, 20200000006209, 20200000061839, 20200000067627, 20200000074495, 20200000103421, 20200000103549, 20200000151577, 20200000193946, 20200000195732, 20200000209655, 20200000222619, 20200000225389, 20200000226773, 20200000227591, 20200000240917, 20200000249727, 20200000367869, 20200000377476, 20200000402800, 20200000405146, 20200000413354, 20200000413804, 20200000422741, 20200000441259, 20200000446113, 20200000448069, 20200000453193, 20200000460517, 20200000463253, 364901807, 374308914, 446054354, 514643583, 524299475, 547001249, 548580856, 618790117, 625311640, 657365840, 743974925, 804637783, 952658953, 972569888, 985443502, 1010600386, 1017551780, 1085899628, 1494487883, 20200000002157, 20200000036354, 20200000038308, 20200000051831, 20200000054438, 20200000056982, 20200000059481, 20200000061820, 20200000071760, 20200000089367, 20200000122900, 20200000123087, 20200000144366, 20200000155372, 20200000159271, 20200000168783, 20200000187098, 20200000194933, 20200000199196, 20200000206261, 20200000246012, 20200000260919, 20200000269121, 20200000280829, 20200000356426, 20200000365999, 20200000369194, 20200000378988, 20200000384688, 20200000391716, 20200000396449, 20200000400202, 20200000403838, 20200000412176, 20200000413761, 20200000415276, 20200000426497, 20200000428965, 20200000435895, 20200000440479, 20200000444938, 20200000447266, 20200000457054, 20200000461398, 329218549, 363485433, 481589020, 492998445, 510040874, 530911354, 542180079, 546937504, 584721587, 624716716, 724596345, 860822992, 977247771, 1010590042, 1049896711, 1060954956, 1062266490, 1064017845, 1129234419, 1345484398, 20200000061162, 20200000108236, 20200000121520, 20200000135201, 20200000136292, 20200000139008, 20200000144242, 20200000149349, 20200000152012, 20200000159114, 20200000165332, 20200000209650, 20200000221519, 20200000229796, 20200000282057, 20200000355023, 20200000357051, 20200000360773, 20200000365560, 20200000370846, 20200000374739, 20200000391864, 20200000403736, 20200000417232, 20200000438574, 20200000444625, 20200000447603, 20200000454620, 20200000457569, 20200000459691, 20200003182596, 317107309, 352881340, 384680007, 426702017, 427133086, 431213733, 517113843, 532497159, 657638474, 834715612, 899867918, 922318156, 963310052, 991246737, 998071189, 1023889969, 1026909621, 1103310426, 1158888051, 1206275828, 1498180338, 20200000004162, 20200000007121, 20200000016934, 20200000029445, 20200000114425, 20200000115234, 20200000146092, 20200000147854, 20200000153878, 20200000156159, 20200000156966, 20200000172157, 20200000187132, 20200000204293, 20200000274528, 20200000281330, 20200000360779, 20200000362154, 20200000372439, 20200000375392, 20200000404064, 20200000404154, 20200000406968, 20200000411198, 20200000419969, 20200000454056, 20200000460096, 20200002102252, 20200002292659, 20200003202527, 366459988, 382069612, 394026416, 414292283, 428832518, 434826169, 455650341, 471063940, 512727332, 588361631, 616696838, 627224987, 664783704, 842849113, 865212458, 1013585593, 1032443533, 20200000001024, 20200000003394, 20200000014617, 20200000031314, 20200000067780, 20200000077383, 20200000078681, 20200000095668, 20200000113619, 20200000121809, 20200000135774, 20200000136134, 20200000137920, 20200000150802, 20200000173773, 20200000185707, 20200000243982, 20200000245359, 20200000254875, 20200000362096, 20200000377417, 20200000379779, 20200000390089, 20200000394253, 20200000397957, 20200000401330, 20200000431759, 20200000432416, 20200000436429, 20200000448269, 20200000452658, 20200002972491, 306486840, 533630781, 571393580, 571667540, 689165391, 706011041, 734339387, 844570015, 871186704, 949275582, 952468684, 1093774002, 1120630354, 1179748521, 20200000027735, 20200000028333, 20200000030549, 20200000038278, 20200000039536, 20200000043387, 20200000067504, 20200000070040, 20200000071419, 20200000080200, 20200000081829, 20200000117328, 20200000144539, 20200000185025, 20200000185831, 20200000196286, 20200000205072, 20200000215404, 20200000229007, 20200000233979, 20200000240325, 20200000244766, 20200000258479, 20200000281141, 20200000351028, 20200000369251, 20200000390297, 20200000397694, 20200000416739, 20200000434715, 20200000437443, 20200000439544, 20200000446824, 20200000462807, 20200001307244, 20200002150661, 281796108, 323275151, 387428400, 412453843, 442012681, 477147527, 512901414, 535176909, 674657235, 746894884, 848225542, 902970360, 983531009, 1007120869, 1046450453, 1082193416, 1086101495, 1093324156, 1105408313, 1163598050, 1485522726, 20200000028929, 20200000047860, 20200000057577, 20200000058494, 20200000073567, 20200000075114, 20200000079155, 20200000118751, 20200000129378, 20200000131908, 20200000137013, 20200000160526, 20200000165286, 20200000204655, 20200000206042, 20200000221969, 20200000229581, 20200000255879, 20200000256493, 20200000272156, 20200000354044, 20200000382741, 20200000397474, 20200000412690, 20200000422171, 20200000424778, 20200000425359, 20200000430197, 20200000436266, 20200000439107, 20200000439464, 20200000439489, 20200000446029, 20200000447923, 20200000449083, 20200000449769, 20200000450209, 20200000456391, 20200000456917, 20200000458861, 20200002577254, 20200003081221, 364248748, 364733950, 388633565, 411989998, 429688355, 438979179, 451427448, 482819029, 506645343, 536075254, 564572458, 575147772, 585027354, 762144812, 818938912, 879407117, 914258835, 959911978, 960229853, 984950827, 1005523811, 1006878949, 1016431735, 1172052423, 20200000007939, 20200000020897, 20200000034864, 20200000062227, 20200000065303, 20200000079062, 20200000087932, 20200000100357, 20200000128021, 20200000129576, 20200000138921, 20200000144527, 20200000182804, 20200000191091, 20200000203210, 20200000216914, 20200000217092, 20200000226063, 20200000250338, 20200000262476, 20200000383062, 20200000396307, 20200000401860, 20200000402129, 20200000415750, 20200000421107, 20200000422755, 20200000430857, 20200000431910, 20200000437516, 20200000440539, 20200000444614, 20200000459840, 410820257, 527631596, 532107328, 556430416, 560916561, 610277867, 639615384, 775991654, 778368902, 911011722, 934746535, 951937596, 955208599, 962510268, 968999008, 971157983, 1013709482, 1126517945, 1167990790, 1168490261, 1170361068, 1183319088, 20200000006324, 20200000025963, 20200000026710, 20200000029511, 20200000035151, 20200000035337, 20200000037529, 20200000039516, 20200000066006, 20200000092304, 20200000106078, 20200000138601, 20200000173400, 20200000177121, 20200000200052, 20200000235314, 20200000236192, 20200000245611, 20200000247337, 20200000248853, 20200000277464, 20200000355331, 20200000363136, 20200000367902, 20200000373656, 20200000377594, 20200000381459, 20200000384967, 20200000398130, 20200000404716, 20200000409294, 20200000415842, 20200000431807, 20200000444654, 20200000445255, 20200000456215, 20200000458828, 20200002144578, 398456162, 417022736, 440684094, 458511266, 474764934, 540384518, 554973925, 568894377, 586201621, 612268513, 625252125, 659981681, 771885464, 775888026, 847492141, 884923615, 895869909, 932500451, 946506172, 960875436, 985945557, 1027347475, 1096516113, 1119856566, 1126835894, 1136245281, 1143011067, 1147002179, 1148321705, 1205134041, 20200000005845, 20200000037103, 20200000044365, 20200000049710, 20200000056171, 20200000066961, 20200000068151, 20200000090109, 20200000091468, 20200000101068, 20200000108656, 20200000120522, 20200000132846, 20200000148825, 20200000190485, 20200000204827, 20200000211972, 20200000240118, 20200000376269, 20200000385856, 20200000386021, 20200000399449, 20200000407002, 20200000425520, 20200000434895, 20200000438020, 20200000444657, 20200000447444, 20200000459809, 20200002180236, 303337299, 368711345, 476500832, 481623941, 521167370, 537931449, 583555212, 606378030, 687285593, 718421443, 777523908, 883389911, 891194610, 894234843, 959498315, 977678950, 1001555097, 1151030355, 1497712444, 20200000000163, 20200000021422, 20200000021858, 20200000063861, 20200000080677, 20200000082888, 20200000084895, 20200000109835, 20200000110746, 20200000131194, 20200000136951, 20200000170782, 20200000214007, 20200000239688, 20200000255539, 20200000270506, 20200000354671, 20200000379106, 20200000387095, 20200000400691, 20200000424651, 20200000426662, 20200000435987, 20200000441341, 20200000450477, 20200000457557, 20200001832311, 20200002314210, 20200002628330, 392915994, 425456242, 448178425, 459109672, 477120941, 528626975, 587649152, 622944658, 723451259, 756975901, 982805507, 1003614272, 1053533457, 1128664538, 1145240210, 1163664489, 1185429113, 20200000007945, 20200000012071, 20200000023238, 20200000033804, 20200000041735, 20200000051679, 20200000094145, 20200000109178, 20200000116813, 20200000138034, 20200000148102, 20200000181729, 20200000184351, 20200000187561, 20200000217128, 20200000224434, 20200000229081, 20200000231621, 20200000244502, 20200000251356, 20200000262895, 20200000273037, 20200000279445, 20200000357856, 20200000383205, 20200000399964, 20200000401414, 20200000403468, 20200000410326, 20200000421213, 20200000443753, 20200000445004, 20200000446178, 20200000451854, 20200000458584, 20200000633817, 20200001453709, 403684733, 417373312, 500097320, 662839145, 672246969, 748559562, 845225648, 878601569, 882507985, 904737541, 937704674, 951364656, 1072456036, 1117719909, 1132358193, 1141842849, 1177953618, 1185419198, 20200000003284, 20200000043208, 20200000061888, 20200000061970, 20200000066250, 20200000080361, 20200000110705, 20200000111627, 20200000118931, 20200000124531, 20200000140078, 20200000157661, 20200000171031, 20200000171135, 20200000181516, 20200000208177, 20200000220543, 20200000241500, 20200000242747, 20200000248955, 20200000253271, 20200000255012, 20200000269258, 20200000357981, 20200000373381, 20200000377578, 20200000392677, 20200000393910, 20200000403994, 20200000424373, 20200000433968, 20200000437139, 20200000442642, 20200000442674, 20200000456214, 20200000460245, 20200000461268, 20200002102399, 374264521, 391286443, 421998342, 453907609, 480983669, 482688777, 507407813, 563939799, 567533074, 597986893, 739234325, 792966342, 808108897, 860570925, 878287161, 976066160, 988383602, 1051617132, 1156775187, 1163998850, 20200000005156, 20200000014571, 20200000015091, 20200000022253, 20200000024992, 20200000026876, 20200000047200, 20200000104750, 20200000112023, 20200000115561, 20200000127041, 20200000149198, 20200000150140, 20200000152765, 20200000156130, 20200000173541, 20200000223845, 20200000224199, 20200000228273, 20200000233440, 20200000234560, 20200000240932, 20200000244655, 20200000245754, 20200000249365, 20200000274171, 20200000359109, 20200000362204, 20200000371026, 20200000378571, 20200000391754, 20200000408511, 20200000417912, 20200000421171, 20200000437925, 20200000439268, 364387007, 444844790, 475299900, 488143008, 510461758, 528805631, 531325150, 549646589, 741224942, 867570291, 883995626, 948715059, 976688720, 978697315, 1046452451, 1052109508, 1093466319, 1106357603, 1135797048, 1145275343, 1173365557, 1186614242, 20200000007583, 20200000036141, 20200000040075, 20200000047126, 20200000054599, 20200000058259, 20200000059963, 20200000083360, 20200000091504, 20200000109951, 20200000113707, 20200000116588, 20200000127737, 20200000136676, 20200000140994, 20200000165930, 20200000193492, 20200000205029, 20200000228326, 20200000241768, 20200000245845, 20200000257338, 20200000275519, 20200000355602, 20200000362189, 20200000367409, 20200000367416, 20200000392509, 20200000395458, 20200000413290, 20200000422083, 20200000427024, 20200000452874, 20200000455431, 20200000457334, 20200001662091, 20200001822804, 307741537, 399166662, 510673136, 576717926, 577731951, 578463146, 900026725, 977646169, 1034045850, 1076985045, 1077543243, 1081749138, 1100139790, 1116620059, 1185580782, 20200000009496, 20200000044478, 20200000047369, 20200000063168, 20200000077314, 20200000090393, 20200000096925, 20200000099021, 20200000103929, 20200000111340, 20200000124636, 20200000130576, 20200000140123, 20200000177690, 20200000178637, 20200000190531, 20200000192775, 20200000209875, 20200000212806, 20200000239257, 20200000249116, 20200000259310, 20200000278402, 20200000353056, 20200000353800, 20200000359803, 20200000383840, 20200000384016, 20200000384782, 20200000387137, 20200000396155, 20200000409451, 20200000412100, 20200000415816, 20200000428232, 20200000450418, 20200000450986, 20200000453405, 20200000453653, 20200002184198, 363282253, 418900349, 502201666, 577413916, 587366035, 589414661, 634095447, 689936776, 699997830, 709970947, 719883477, 739002910, 783686024, 905294981, 921385179, 936966605, 943915646, 945621521, 1012540379, 1033713849, 1044677336, 1050692630, 1091944550, 1102054727, 1446850955, 20200000001446, 20200000016882, 20200000040533, 20200000041961, 20200000062168, 20200000089797, 20200000111470, 20200000111566, 20200000122589, 20200000164153, 20200000170123, 20200000183220, 20200000187133, 20200000189881, 20200000202770, 20200000206787, 20200000232597, 20200000248545, 20200000252639, 20200000256732, 20200000266205, 20200000374914, 20200000388017, 20200000388421, 20200000435584, 20200000442659, 20200000457682, 20200000462101, 20200002103156, 328205875, 332639548, 363590051, 425194759, 428267291, 475263094, 595649238, 608197113, 634088302, 668159524, 669398769, 714796093, 719169017, 740737088, 836558771, 950710606, 1021975089, 1073594113, 1080876046, 1104837211, 1148610566, 20200000001570, 20200000003453, 20200000028650, 20200000035125, 20200000054312, 20200000084365, 20200000088062, 20200000107483, 20200000121195, 20200000133211, 20200000159021, 20200000167288, 20200000171306, 20200000187119, 20200000195152, 20200000198919, 20200000207079, 20200000223935, 20200000224876, 20200000230773, 20200000259299, 20200000270053, 20200000278334, 20200000379327, 20200000382444, 20200000401556, 20200000404252, 20200000408754, 20200000414806, 20200000415129, 20200000415604, 20200000422871, 20200000423788, 20200000423948, 20200000431487, 20200000438761, 20200000439749, 20200000442326, 20200000443085, 20200000454449, 20200000455444, 20200000459674, 20200000460529, 20200000460577, 308629295, 341595032, 369219208, 442970576, 457446957, 463416998, 486269920, 489615659, 603109324, 616073484, 824793454, 829587759, 837274884, 857449903, 952715194, 1079253434, 1079799899, 1132473553, 1194969046, 20200000014964, 20200000015520, 20200000032785, 20200000036313, 20200000037969, 20200000043829, 20200000049411, 20200000050923, 20200000064368, 20200000076081, 20200000086217, 20200000119710, 20200000121505, 20200000128113, 20200000143443, 20200000152313, 20200000177010, 20200000209943, 20200000212980, 20200000254245, 20200000257389, 20200000262189, 20200000278210, 20200000278352, 20200000279210, 20200000283195, 20200000363622, 20200000370366, 20200000378516, 20200000435863, 20200000449468, 20200000450402, 20200000463942, 20200000557224, 304871622, 368677368, 384334005, 456327713, 502473222, 524624575, 524731580, 648652228, 656885484, 775402833, 904209370, 973616104, 997675942, 1045901853, 1056681903, 1059056681, 1065751059, 20200000021739, 20200000041891, 20200000051959, 20200000055001, 20200000055605, 20200000070209, 20200000099890, 20200000101287, 20200000107276, 20200000120315, 20200000175230, 20200000232592, 20200000250170, 20200000262162, 20200000276052, 20200000355752, 20200000363600, 20200000375604, 20200000382202, 20200000395973, 20200000407123, 20200000420077, 20200000420255, 20200000425923, 20200000437459, 20200000444179, 20200000461285, 334833057, 417634504, 441903971, 455680974, 491075156, 503098602, 547109049, 549105915, 590380956, 597208690, 639970453, 652862352, 766504466, 777707141, 864648152, 869329921, 880178264, 901999406, 971617423, 1046597628, 1055455919, 1066091585, 1098157959, 1218127426, 1450700255, 20200000070113, 20200000083840, 20200000103950, 20200000119418, 20200000126829, 20200000140380, 20200000144038, 20200000162777, 20200000187834, 20200000202036, 20200000232858, 20200000240219, 20200000240859, 20200000247150, 20200000261334, 20200000365777, 20200000366351, 20200000367223, 20200000373597, 20200000380530, 20200000388821, 20200000390327, 20200000414409, 20200000424620, 20200000431265, 20200000441730, 20200000451000, 20200000455381, 20200003224919]


In [0]:

publisher_not_in_db = [ str(x) for x in publisher_not_in_list ]


In [0]:

20200003224919 in [publisher_not_in_db]




In [0]:

type_2 = [1053341907,1303477251,1352778019,1370800733,1424127493,1436507807,1464836192,1466203839,1471613168,1473796954,1476239364,1476517528,1483441376,1490065786,1492883229,1494463403,1494556166,1494652780,1494680273,1494689023,1494690321,1494692072,1494692878,1494692967,1494711326,1494711977,1494713074,1494726685,1494730318,1494780334,1494807865,1494810446,1494816942,1494821603,1494824513,1494825069,1494826782,1494826858,1494832125,1494835122,1494835618,1494835776,1494842734,1494846888,1494848359,1494850474,1494857776,1494859873,1494860148,1494865453,1494867061,1494867159,1494868673,1494869059,20200003316001,20200003414528,20200003414531,20200003414561,20200003414573,20200003414576,20200003414618,20200003414653,20200003414654,20200003414672,20200003414781,20200003414814,20200003414822,20200003414829,20200003414836,20200003414879,20200003414882,20200003414916,20200003414949,20200003414979,20200003415181,20200003415292,20200003415519,20200003415562,20200003415585,20200003415605,20200003415620,20200003415654,20200003415676,20200003415713,20200003415714,20200003415725,20200003415730]
typ_2_list = [ str(x) for x in type_2 ]

In [0]:

from aadatapipelinecore.core.utils.date_ import to_date, str_date
def read_raw_data(spark,date):
    parquet_base = "s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/"
    csv_base = "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/"
    # parquet files
    boundary = to_date('2019-07-14')
    real_date = to_date(date)
    if real_date >= boundary:
        path = parquet_base + "date=" + date + "/"
        df = spark.read.option("basePath", parquet_base).parquet(path)
        return df
    # csv files
    else:
        path = csv_base + date + "/*/sbe_est_publisher/*/*.gz"
        source_df = spark.read.option("delimiter", "\t").csv(path)
        df = source_df.select(f.col("_c0").alias("store_id"),
                f.col("_c1").alias("date"),
                f.col("_c2").alias("platform_id"),
                f.col("_c3").alias("vertical"),
                f.col("_c4").alias("feed"),
                f.col("_c5").alias("id"),
                f.col("_c6").alias("est"),
                f.col("_c7").alias("category_id"),
                f.col("_c8").alias("rank"))
        return df

df = read_raw_data(spark, "2020-01-13")
df.createOrReplaceTempView("temp")

spark.sql("select * from temp where id in ({}) and rank<=1000 and feed <1000  order by rank asc ".format(",".join(typ_2_list))).show(10)

# spark.sql("select platform_id, feed , min(rank), max(rank), Cast(avg(rank) AS int) from temp where store_id = 0 and id in ({}) and rank<=1000 and feed <1000 group by platform_id, feed order by min(rank)  ".format(",".join(typ_2_list))).show(2000)


In [0]:

import datetime
from aadatapipelinecore.core.utils.date_ import to_date, str_date
def read_raw_data(spark,date):
    parquet_base = "s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/"
    csv_base = "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/"
    # parquet files
    boundary = to_date('2019-07-14')
    real_date = to_date(date)
    if real_date >= boundary:
        path = parquet_base + "date=" + date + "/"
        df = spark.read.option("basePath", parquet_base).parquet(path)
        return df
    # csv files
    else:
        path = csv_base + date + "/*/sbe_est_publisher/*/*.gz"
        source_df = spark.read.option("delimiter", "\t").csv(path)
        df = source_df.select(f.col("_c0").alias("store_id"),
                f.col("_c1").alias("date"),
                f.col("_c2").alias("platform_id"),
                f.col("_c3").alias("vertical"),
                f.col("_c4").alias("feed"),
                f.col("_c5").alias("id"),
                f.col("_c6").alias("est"),
                f.col("_c7").alias("category_id"),
                f.col("_c8").alias("rank"))
        return df

start = "2020-03-01"
end = "2020-04-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)
new_df = []
for d in dates:
    print d
    df = read_raw_data(spark, d)
    df.createOrReplaceTempView("temp")
    new_df.extend(spark.sql("select * from temp where id in ({}) and rank<=1000 and feed <1000 order by rank asc ".format(",".join(publisher_not_in_db))).collect())


In [0]:

print spark.createDataFrame(new_df).createOrReplaceTempView("temp")

# spark.sql("select * from temp ").show(80)
spark.sql("select count(1) from temp ").show()
spark.sql("select count(distinct id) from temp ").show()
spark.sql("select min(rank) , max(rank) , cast(avg(rank) AS int) from temp ").show()

In [0]:

from pyspark.sql import functions as F
df_dna = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-06-30/").cache()
df_event = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-publisher-event.v1/dimension/update_date=2020-06-30/").cache()

df_dna.createOrReplaceTempView("dna_logging")
spark.sql("select * from dna_logging where start_date between '2020-03-01' and '2020-03-31' ").show(3)


In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2  -U app_tomcat -d aa_android -p 5433 << EOF

BEGIN;

SELECT 
    *
    --count(1)
    --id, app_id, store_id, date, type, old_value, new_value
FROM event
WHERE type = 33 
LIMIT 10;

rollback;

EOF

In [0]:


sql_where = "update_date = '2020-06-22'"
dna_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/").where(sql_where)
dna_df.createOrReplaceTempView("dna_table")
spark.sql("select * from dna_table where app_id=284945681").show()


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- universal_id : company id
-- parent_id : parent id
-- select * from dna_universal_company_parent where parent_id in (1000200000000033) and disabled='f' and confirmed='t' limit 1000;
-- select * from dna_universal_company_parent where start_date < '2012-01-01' limit 10;



-- universal_id : company id
-- company id : publisher id
select * from dna_universal_company_mapping where company_id=1521782070   limit 100;
--select * from dna_universal_company_parent where universal_id=1000200000230577 limit 10;
--select * from dna_universal_company_parent where universal_id=1000200000039960 limit 10;

--select * from dna_genre_id_product_mapping  limit 5;

-- select * from in_unified_product_product_mapping  limit 3;
-- select * from in_app_dna_info_mapping limit 3;

-- select * from dna_genre_mapping where parent_genre_id is null limit 5;
-- select * from dna_universal_company_office limit 3

-- \d dna_universal_company_office
-- select * from dna_universal_company_office where type != 'HQ' limit 3

EOF
